In [ ]:
import numpy as np
#%matplotlib nbagg
#%matplotlib inline

import  matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import pandas as pd

plt.rcParams['figure.figsize'] = [10, 8]
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (7, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rc('font', family='serif')
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')
pylab.rcParams.update(params)

#plt.rcParams['xtick.color']='white'
#plt.rcParams['ytick.color']='white'
#plt.rcParams['axes.labelcolor']='white'
#plt.rcParams['axes.titlecolor']='white'
#plt.rcParams['axes.facecolor']='white'
#%matplotlib notebook

import statistics as stats
from scipy.optimize import curve_fit
from os import remove
import os #para trabajar con directorios y carpetas
import os.path #para determinar si existe una carpeta
import shutil #para eliminar carpetas que tengan archivos adentro o mover archivos
from scipy.interpolate import CubicSpline
from scipy import interpolate
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

#from astroML.plotting import setup_text_plots
#setup_text_plots(fontsize=15, usetex=False)

#ax.tick_params(direction='in', length=10, width=2, colors='black',
 #      bottom=1, top=1, left=1, right=1,)



In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
import os.path
import time

#notebook_name = "Funciones_colab.ipynb" # Reemplaza "nombre_del_notebook.ipynb" con el nombre de tu notebook
#notebook_path = os.path.abspath(notebook_name)
#notebook_path = os.path.join(os.getcwd(),'Codes/Funciones_colab.ipynb')#os.path.abspath("Codes/Funciones_colab.ipynb")

#print("Último guardado:", time.ctime(os.path.getmtime(notebook_path)))



In [ ]:
import importlib
import subprocess

def import_with_install(package):
    """
    Intenta importar un paquete, y si falla, lo instala con pip3
    """
    try:
        if package =="POT":
            importlib.import_module("ot")
        else:
            importlib.import_module(package)

    except ImportError:
        print(f"{package} no está instalado, se procederá a instalarlo...")
        try:
            subprocess.check_call(['pip3', 'install', package])
        except subprocess.CalledProcessError:
            print(f"No se pudo instalar {package}. Por favor, instale el paquete manualmente.")
            return
        print(f"{package} se ha instalado correctamente.")
    finally:
        if package =="POT":
            globals()[package] = importlib.import_module("ot")
        else:
            globals()[package] = importlib.import_module(package)

import_with_install('POT')
import_with_install('IPython')
#import_with_install('specutils')
from ot import *

In [ ]:
def remove_filter(name,FILTRO,filtro):
    for i in range(len(filtro)):
        if filtro[i]==name:
            #c.append(i)
            FILTRO.pop(i)
            filtro.pop(i)
            break

def remove_spec(name):
     for i in range(len(spec)):
        if spec[i]==name:
            #c.append(i)
            ESPECTRO.pop(i)
            spec.pop(i)
            DJ_SPEC.pop(i)
            datos_espectros_spline.pop(i)
            break

def rms(x,parametros):
    #print(x)
    promedio=np.mean(x)
    #promedio
    residuales=x-promedio
    l=residuales**2
    #print(residuales)
    val=np.sqrt(sum(l)/(len(x)-parametros))
    return val

def L1(x):
    mediana=np.median(x)
    residuales=abs(x-mediana)
    val=np.median(residuales)
    return val



def maximun_light_curve(FILTRO,filtro): #arrelgo con los datos de todos los filtros, el nombre de los filtros
    plt.style.use('classic')
    maximos=[]
    for j in range(len(FILTRO)):
        like=1
        while like==1:
            DJ=[]
            flux=[]
            fluxerr=[]
            for i in range(len(FILTRO[j])):

                DJ.append(np.float(FILTRO[j][i][0]))
                flux.append(np.float(FILTRO[j][i][1]))


                #necesitamos las barras de error
                fluxerr.append(np.float(FILTRO[j][i][2]))
            xerr=np.zeros(len(flux)) #porque no hay error en X



            plt.errorbar(DJ, flux, fluxerr,xerr,'.','g')
            plt.grid()
            plt.xlabel('Dia Juliano')
            plt.ylabel('FLUJO')
            plt.title('Curva de Luz filtro '+str(filtro[j]))
            plt.show()

            #-----------------------------------------
            #primero creamos el dia juliano para el polyfit

            JDpl=[]
            fluxpl=[]
            flux_errpl=[]
            a=0
            print('AHORA SE PROCEDE A HACER EL FIT AL REDEDOR DEL MAXIMO DE LA SUPER NOVA')
            JDpl_menor=float(input('Ingrese cota inferior para el polifit   '))
            JDpl_mayor=float(input('Ingrese cota superior para el polifit   '))
            while a==0:
                for i in range(len(DJ)):

                    if DJ[i] >= JDpl_mayor:
                            break
                    if DJ[i]>=JDpl_menor:
                        JDpl.append(DJ[i])
                        fluxpl.append(flux[i])
                        flux_errpl.append(fluxerr[i])

                a=len(JDpl)
                if a==0:
                    print('El rango escogido no tiene valores ')
                    JDpl_menor=float(input('Ingrese cota inferior para el polifit   '))
                    JDpl_mayor=float(input('Ingrese cota superior para el polifit   '))

            #sacamos los weights------------------------------------
            W=[]
            for i in range(len(flux_errpl)):
                w=np.float(1/flux_errpl[i])
                W.append(w)



            par2=np.polyfit(JDpl,fluxpl,2,w=W)


            #--------------------------------------------------
            #calculamos el maximo con el criterio de la 2da derivada

            #para gradoo 2 es mas facil:
            DJ_max2=-par2[1]/(2*par2[0])
            print('EL dia juliano del maximo para grado 2 es: ',DJ_max2)
            Flux_max2=par2[0]*DJ_max2**2+par2[1]*DJ_max2+par2[2]
            print('EL Flujo maximo para grado 2 es: ',Flux_max2)

            #---------------------------------------------------
            #para grado 3 tenemos dos soluciones porque el punto cero me da una formula cuadratica

            par3=np.polyfit(JDpl,fluxpl,3,w=W)
            a=par3[0]
            print(type(a),'el tipo de a')
            b=par3[1]
            c=par3[2]
           # x1=np.float((-2*par3[1]+((-2*par3[1])**2-(4*3*par3[0]*par3[2]))**0.5)/(2*3*par3[0]))
            x1=np.float((-2*b+((-2*b)**2-(4*3*a*c))**0.5)/(2*3*a))

            x2=(-2*par3[1]-((-2*par3[1])**2-(4*3*par3[0]*par3[2]))**0.5)/(2*3*par3[0])

            #print ('el valor de x1 es	:',x1)
            #print ('el valor de x2 es	:',x2)

            signo=6*par3[0]*x1+2*par3[1]
            signo2=6*par3[0]*x2+2*par3[1]
            print('x1',x1)
            print(type(x1))
            print('x2 ',x2)
            print('signo' ,signo)
            print('signo2 ',signo2)

            global DJ_max3

            if signo==' nan':
                DJ_max3=DJ_max2
                print('No tiene maximo en polinomio grado 3, por lo que es el mismo para grado 2',DJ_max3)


            elif signo<0:
                #print ('x1 es un maximo')
                DJ_max3=x1
                print ('EL dia juliano del maximo para grado 3 es	:',DJ_max3)

            elif signo2<0:
                #print ('x2 es un maximo')
                DJ_max3=x2
                print ('EL dia juliano del maximo para grado 3 es',DJ_max3)


            Flux_max3=par3[0]*DJ_max3**3+par3[1]*DJ_max3**2+par3[2]*DJ_max3+par3[3]
            print('EL Flujo maximo para grado 3 es: ',Flux_max3)


            #-----------------------------------------------------------------------------

            fig1=plt.figure(1)
            plt.figure(figsize=(6,5))
            xp=np.linspace(np.min(JDpl),np.max(JDpl),len(JDpl))
            plt.plot(xp,np.polyval(par2,xp),color='orange',label='pol grado 2')
            plt.plot(xp,np.polyval(par3,xp),'--',color='green',label='pol grado 3')



            plt.scatter(JDpl,fluxpl)
            plt.grid()
            plt.xlabel('Dia Juliano')
            plt.ylabel('FLUJO')
            plt.title('Curva de Luz filtro '+str(filtro[j]))
            plt.legend()
            plt.show()



            #------------------------------------------------------------------
            DJ_zoom=[]
            flux_zoom=[]

            for i in range(len(DJ)):
                if abs(DJ_max3-DJ[i])<=50:
                    DJ_zoom.append(DJ[i])
                    flux_zoom.append(flux[i])
            #------------------------------------------------------------------

            fig2=plt.figure(2)
            plt.figure(figsize=(6,5))
            xp=np.linspace(np.min(JDpl),np.max(JDpl),len(JDpl))
            plt.plot(xp,np.polyval(par2,xp),color='orange',label='pol grado 2')
            plt.plot(xp,np.polyval(par3,xp),'--',color='green',label='pol grado 3')



            plt.scatter(DJ_zoom,flux_zoom)
            plt.grid()
            plt.xlabel('Dia Juliano')
            plt.ylabel('FLUJO')
            plt.title('Curva de Luz filtro '+str(filtro[j]))
            plt.legend()
            plt.show()



            print('#########################################################################')



            #-----------------------------------------------------
            like=float(input('Ajuste correcto? o desea cambiar las fechas?: 1=cambiar 2=conservar '))
            while(like!=1 and like!=2):
                like=float(input('Ingrese 1 o 2: '))




            if like ==2:
                guardar=float(input('Desea guardar el maximo de grado 2 o 3:  ' ))
                while(guardar!=2 and guardar!=3):
                    guardar=float(input('Ingrese 1 o 2: '))
                if guardar==2:
                    DJ_max=DJ_max2
                elif guardar==3:
                    DJ_max=DJ_max3


                xp=np.linspace(np.min(JDpl),np.max(JDpl),len(JDpl))
                eje_y=np.polyval(par2,xp)

                file=open('Ajuste_curva_'+str(filtro[j]),'w')
                #guardo el ajuste de la curva en un archivo de texto
                file.write('#DATOS CURVA AJUSTADA\n')
                for i in range(len(xp)):
                    file.write(str(xp[i])+'\t'+str(eje_y[i])+'\n')

                file.write('\n')
                file.write('#DATOS A LOS CUALES SE AJUSTA LA CURVA\n')

                for ii in range(len(JDpl)):
                    file.write(str(JDpl[ii])+'\t'+str(fluxpl[ii])+'\n')
                file.close()




                maximos.append(DJ_max)
    return maximos


def data_curvas(path,rename=False): #funcion para leer los archivos de texto de los filtros
    import pandas as pd
    with open(path,'rt') as f2:
        lineas= [linea.split() for linea in f2]
    cantidad_filtros=0 #contador de filtros
    filtro=[] #arreglo con los filtros
    componente_filas=[] #me dice las componentes de las lineas en que estan los datos por filtro (ejemplo: filtro B de la 7
    for i in range(len(lineas)):
        if len(lineas[i])>=2:
            if lineas[i][1]=='FILTER':
                #print (lineas[i][2])
                cantidad_filtros=cantidad_filtros+1
                filtro.append(lineas[i][2])
                componente_filas.append(i)


    #print ('HAY UN TOTAL DE ',cantidad_filtros,' FILTROS')
    #print (filtro)
    #print (componente_filas)

    FILTRO=[]
    for j in range(len(componente_filas)):
        nombre = "filtro_" + str(filtro[j])
        #print (nombre)
        filtro1=[]
        if j==len(componente_filas)-1:
            final=len(lineas)
            lineas_filtro=np.arange(componente_filas[j]+2,final,1)
            #print (lineas_filtro)
            for i in lineas_filtro:
                filtro1.append(lineas[i])
                filtro1.append
        else:
            lineas_filtro=np.arange(componente_filas[j]+2,componente_filas[j+1]-1,1) #me dice las lineas que tengo q guardar por filtro
            #print (lineas_filtro)
            for i in lineas_filtro:
                filtro1.append(lineas[i])

        filtro1=pd.DataFrame(filtro1)
        #print(filtro1)
        filtro1[0] = filtro1[0].astype(float)
        filtro1[1] = filtro1[1].astype(float)
        filtro1[2]=filtro1[2].astype(float)

        if rename==True:
            filtro1 = filtro1.rename(columns={0: 'mjd', 1: 'flux'})

        FILTRO.append(filtro1)
        #print (filtro1)
    return FILTRO,filtro #me retorna FILTRO donde estan los datos y filtro que es una lista con los filtros

def GP(x,y,yerr,evaluar): #A la funcion se le ingresan los arreglos que se le realizara el proceso gaussiano, y donde quiero evaluar
    from sklearn import gaussian_process
    from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
    from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel)
    kernel1=1.0 * RBF(length_scale=10,length_scale_bounds=(1e-20, 100000.0))     #Defino el kernel a utilizar

    kernel3= Matern(length_scale=1.0, nu=1.5, length_scale_bounds=(1e-5, 1e5))

    #-------------------Aplicamos el proceso Gaussiano-----------------------
    X = x.reshape(-1, 1)                    #Redefinimos las dimensiones de X para que quede como 2d
    X.shape

    gp = gaussian_process.GaussianProcessRegressor(kernel=kernel1*kernel3, n_restarts_optimizer=9,alpha=yerr**2)  #llamamos la funcion del Gaussian Process
    gp.fit(X, y)
    #----------------------------------------------------------------------------------
    #ahora creo los arreglos donde voy a evaluar mi proceso Gaussiano.

    x_pred=np.array(evaluar).reshape(-1,1)

    #x_pred2=np.linspace(min(x),max(x),1000000).reshape(-1,1)#por si quiero los intervalos de confianza en un futuro
    #y_pred2,sigma2=gp.predict(x_pred2,return_std=True)

    y_pred,sigma=gp.predict(x_pred,return_std=True)
    return x_pred,y_pred,sigma


def best_reg(A,M,weights):#entran todos los parametros que quiero calcular
    #This function look for the best regularization term on the computation of the barycenter
    # If the interpolation cant be done, this increase the regularization term
    import math
    import ot
    global reg_def
    a=np.arange(-7,-1,0.1)#-7,10,1 /-3,-2,1
    for i in range(len(a)):
        reg1=5*10**(a[i])
        #print('regularization term:',reg1)
        #you can choose use another method
        #bary_wass1=ot.bregman.barycenter(A,M,reg1,weights,verbose=False,method='sinkhorn_stabilized',tau=1e250)
        bary_wass1=ot.bregman.barycenter(A,M,reg1,weights,verbose=False,method='sinkhorn',numItermax =10)

        cont=0
        #print(bary_wass1)
        for j in range(len(bary_wass1)):
            if math.isnan(bary_wass1[j])==True or math.isinf(bary_wass1[j])==True:
                  cont=cont+1
        #print('a,index,cont,bary_wass:',a[i],i,cont,bary_wass1)
        if cont==0:
            reg_def=reg1
            break
        else:
            #print(a[i])
            reg_def=9999999999


    return reg_def,bary_wass1

def OT(a1,a2,alpha):#entran dos distribuciones y la cercanica de cada una
    #%% parameters
    import ot

    n = len(a1)  # nb bins

    # bin positions
    x = np.arange(n, dtype=np.float64)

    # creating matrix A containing all distributions
    A = np.vstack((a1, a2)).T
    n_distributions = A.shape[1]

    # loss matrix + normalization
    M = ot.utils.dist0(n)
    #print(M)
    M /= M.max()
    #print(M)

    #%% barycenter computation

    #alpha = cercania#0.5  # 0<=alpha<=1
    weights = np.array([ 1-alpha, alpha])

    # l2bary
    #bary_l2 = A.dot(weights)

    # regularized wasserstein

    reg,bary_wass=best_reg(A,M,weights)
    #reg = 1*10**(-3) #mientras mas pequeño este numero parece mejorar el ajuste..
    #ot.tic()
    #bary_wass = ot.bregman.barycenter(A, M, reg, weights)
    #ot.toc()

    # linear programming barycenter
    #ot.tic()
    #bary_wass2 = ot.lp.barycenter(A, M, weights, solver='interior-point', verbose=True)
    #ot.toc()
    return bary_wass

In [ ]:
#Ahora tengo que aplicar la fotometria sintetica
#vamos a crear una funcion para calcular la fotometria sintetica

def Syntetic_photometry(xspec,yspec,xband,yband): #En Amstrong
        contador=0
        S_lambda=[] #Flujo del espectro en la interseccion
        F_lambda=[] #flujo de la banda en la interseccion
        x_lambda=[]

        componente_spectra=[]
        componente_band=[]
        xspec=np.array(xspec)
        yspex=np.array(yspec)
        xband=np.array(xband)
        yband=np.array(yband)
        #xspec=xnew
        for z in range(len(xband)):
            if xband[z] in xspec:
                x_lambda.append(xband[z]) #esta es la lista con las longitudes de onda que se interceptan
                contador=contador+1


        #----------------------calculo de la interseccion del espectro-------------------------------
        for zz in range(len(xspec)):
            for m in range(len(x_lambda)):
                if x_lambda[m]==xspec[zz]:
                    componente_spectra.append(zz)


        for componente in componente_spectra:
            S_lambda.append(yspec[componente])

        #----------------------calculo de la interseccion de la banda--------------------------------
        for cc in range(len(xband)):
            for m in range(len(x_lambda)):
                if x_lambda[m]==xband[cc]:
                    componente_band.append(cc)


        for componente in componente_band:
            F_lambda.append(yband[componente])

        #-------------------------------------------------------

        area_norm_tot=0
        area_band_tot=0
        # AHORA VAMOS A CALCULAR EL AREA DE LA RESPUESTA PARA NORMALIZAR

        #-------FORMA1---------------------------------
        #for ii in range(len(x_lambda)): #area para normalizar
            #area_normalizar=1*F_lambda[ii]*x_lambda[ii]
            #area_norm_tot=area_normalizar+area_norm_tot
        #-------FORMA2----------------------------
        for ii in range(len(xband)):
            area_normalizar=1*yband[ii]*xband[ii]
            area_norm_tot=area_normalizar+area_norm_tot

            #--------para sacar porcentaje--------
            area_band=1*yband[ii]#Esta es sin mutiplicar la longitud de onda para sacar porcentaje
            area_band_tot=area_band+area_band_tot

        #---Ahora calculamo el area de la interseccion
        areatotal1=0
        area_inter_banda_total=0
        for l in range(len(x_lambda)): #area de la banda
            #b1=x_lambda[l]
            #b2=x_lambda[l+1]
            #base=b2-b1
            #h1=F_lambda[l]*S_lambda[l]
            #h2=F_lambda[l+1]*S_lambda[l+1]
            #prom=(h1+h2)/2
            area=1*(F_lambda[l]*S_lambda[l])*x_lambda[l]
            areatotal1=area+areatotal1

            #------para sacar porcentaje-----------
            area_inter_banda=1*F_lambda[l]
            area_inter_banda_total=area_inter_banda_total+area_inter_banda

        print(area_inter_banda_total,area_band_tot)
        porcentaje= 100*(area_inter_banda_total/area_band_tot)
        print('El filtro cubre un ',porcentaje,'%')

        total=areatotal1/area_norm_tot#*areatotal2
        print('num|den')
        print(areatotal1,area_norm_tot)

        return total,porcentaje

In [ ]:
def Syntetic_photometry_v2(xspec,yspec,xband,yband):
    #recibe pandas dataframe o arrays, y calcula la inrterseccion
    #pueden ser de distinto largo y la respuesta de la banda se adapta a la grilla del espectro con regrid

    from sklearn.metrics import auc

    ######################################################################
    # AHORA VAMOS A CALCULAR EL AREA DE LA RESPUESTA PARA NORMALIZAR
    a_banda=auc(xband,yband) #esta es el area de la banda

    #necesitamos el area de la interseccion##############################
    import scipy
    from scipy.interpolate import griddata
    regrid=scipy.interpolate.griddata(xband, yband,xspec, method='linear', rescale=False)
    df_regrid=pd.DataFrame({'wave':xspec,'response':regrid})
    df_regrid = df_regrid.dropna(how='any') #eliminamos los nan

    try:
        a_inter=auc(df_regrid['wave'],df_regrid['response'])
    except ValueError:
        return 0,0
    #####################################################################
    porcentaje=a_inter/a_banda
    #print(a_inter,a_banda)
    #print('porcentaje:',a_inter/a_banda)
    spec1_df=pd.DataFrame({'wave':xspec,'flux':yspec})
    #ahora calculamos el numerador de la integral
    merge = pd.merge(spec1_df, df_regrid, how='inner', on=['wave']) #mismo q df_regrid pero con el flujo del spec

    num=auc(merge['wave'],merge['flux']*merge['response']*merge['wave'])

    #calculamos el denominador

    den=auc(xband,yband*xband) #este es el numerador para normalizar

    total_=num/den
    #print('area :',total_, 'porcentaje:',porcentaje)
    #print('num|den')
    #print(num,den)
    

    return total_,porcentaje

In [ ]:
def planckiana(lambda_,T,k):        #el k es una constante de normalizacion lambda debe estar en micrometros
    import math
    h=6.626070150e-34   #En J*s      #6.626070150e-27 # e ergs*s
    c=3e8                       #velocidad de la luz en m*s
    cte= 1.380649e-23  #en J/K      #1.380649e-16 #constante de boltzman en ergs/K

    numerador=(2.0*h*(c**2))
    denominador=(lambda_**5)*(np.exp(h*c/(lambda_*cte*T))-1)
    B_lambda=numerador/denominador
    return k*B_lambda

In [ ]:
def effective_wavelength(xspec,yspec,xband,yband):
        contador=0
        S_lambda=[] #Flujo del espectro en la interseccion
        F_lambda=[] #flujo de la banda en la interseccion
        x_lambda=[]

        componente_spectra=[]
        componente_band=[]

        #xspec=xnew
        for z in range(len(xband)):
            if xband[z] in xspec:
                x_lambda.append(xband[z]) #esta es la lista con las longitudes de onda que se interceptan
                contador=contador+1


        #----------------------calculo de la interseccion del espectro-------------------------------
        for zz in range(len(xspec)):
            for m in range(len(x_lambda)):
                if x_lambda[m]==xspec[zz]:
                    componente_spectra.append(zz)


        for componente in componente_spectra:
            S_lambda.append(yspec[componente])

        #----------------------calculo de la interseccion de la banda--------------------------------
        for cc in range(len(xband)):
            for m in range(len(x_lambda)):
                if x_lambda[m]==xband[cc]:
                    componente_band.append(cc)


        for componente in componente_band:
            F_lambda.append(yband[componente])

        #-------------------------------------------------------

        area_norm_tot=0
        area_band_tot=0


        #---Ahora calculamo el area de la interseccion
        areatotal1=0
        area_inter_banda_total=0
        for l in range(len(x_lambda)): #area de la banda
            #b1=x_lambda[l]
            #b2=x_lambda[l+1]
            #base=b2-b1
            #h1=F_lambda[l]*S_lambda[l]
            #h2=F_lambda[l+1]*S_lambda[l+1]
            #prom=(h1+h2)/2
            area=1*(F_lambda[l]*S_lambda[l])*x_lambda[l]
            areatotal1=area+areatotal1

            area_normalizar=1*(F_lambda[l]*S_lambda[l])
            area_norm_tot=area_normalizar+area_norm_tot
            #----------para sacar porcentaje
            area_inter_banda=1*F_lambda[l]
            area_inter_banda_total=area_inter_banda_total+area_inter_banda

        # AHORA VAMOS A CALCULAR EL AREA DE LA RESPUESTA PARA NORMALIZAR

        #-------FORMA1---------------------------------
        #for ii in range(len(x_lambda)): #area para normalizar
            #area_normalizar=1*F_lambda[ii]*x_lambda[ii]
            #area_norm_tot=area_normalizar+area_norm_tot
        #-------FORMA2----------------------------
        for ii in range(len(xband)):
            #area_normalizar=1*(F_lambda[l]*S_lambda[l])
            #area_norm_tot=area_normalizar+area_norm_tot

            #-----------
            area_band=1*yband[ii]#Esta es sin mutiplicar la longitud de onda para sacar porcentaje
            area_band_tot=area_band+area_band_tot

        print(areatotal1,area_norm_tot)
        #print(area_inter_banda_total,area_band_tot)
        porcentaje= 100*(area_inter_banda_total/area_band_tot)
        print('El filtro cubre un ',porcentaje,'%')

        total=areatotal1/area_norm_tot#*areatotal2
        print(total)
        return total,porcentaje

In [ ]:

def effective_wavelength_same_length(xspec,yspec,xband,yband):

        areatotal1=0
        area_inter_banda_total=0
        area_normalizar=0
        area_norm_tot=0

        xspec=np.array(xspec)
        yspex=np.array(yspec)
        xband=np.array(xband)
        yband=np.array(yband)

        for l in range(len(xspec)): #area de la banda
            area=1*(yspec[l]*yband[l])*xspec[l]
            areatotal1=area+areatotal1

            area_normalizar=1*(yspec[l]*yband[l])
            area_norm_tot=area_normalizar+area_norm_tot
            #----------para sacar porcentaje
            #area_inter_banda=1*F_lambda[l]
            #area_inter_banda_total=area_inter_banda_total+area_inter_banda

        print(areatotal1,area_norm_tot)
        #print(area_inter_banda_total,area_band_tot)
        #porcentaje= 100*(area_inter_banda_total/area_band_tot)
        #print('El filtro cubre un ',porcentaje,'%')

        total=areatotal1/area_norm_tot#*areatotal2
        print(total)
        return total

In [ ]:
def read_spec(path):
    with open(path,'r') as f2:
        espectro_lineas= [linea.split() for linea in f2]

    componente_filas_spec=[]
    spec=[]
    fases=[]

    for i in range(len(espectro_lineas)):
        if len(espectro_lineas[i])>1:
            if 'SPEC' in espectro_lineas[i][1]:
                componente_filas_spec.append(i)  #me dice en que componente esta el spectro de la posicion
                fase=espectro_lineas[i][2]
                fase=float(fase.strip('PHASE='))
                spec.append(float('%.3f'%fase))
                fases=spec

    ESPECTRO=[]
    for j in range(len(componente_filas_spec)):
        nombre_spec = str(spec[j])
        print (nombre_spec)
        espectro1=[]
        if j==len(componente_filas_spec)-1:
            final=len(espectro_lineas)
            lineas_espectro=np.arange(componente_filas_spec[j]+2,final,1)
            #print (lineas_espectro)
            for i in lineas_espectro:
                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])

        else:
            lineas_espectro=np.arange(componente_filas_spec[j]+2,componente_filas_spec[j+1]-1,1) #me dice las lineas que tengo q guardar por filtro
            #print (lineas_espectro)
            for i in lineas_espectro:

                if len(espectro_lineas[i])==0:
                    break

                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])
        ESPECTRO.append(espectro1)
    #print(fases)
    return ESPECTRO,fases

In [ ]:
def leer_spec(path,ot=False,MJD=False,as_pandas=False,compress=False):

    if compress==True:
        import gzip

        with gzip.open(path, 'rb') as f2:
            espectro_lineas= [linea.split() for linea in f2]
    else:
        with open(path,'rt') as f2:
            espectro_lineas= [linea.split() for linea in f2]

    componente_filas_spec=[]
    spec=[]
    fases=[]
    iniciales=[]
    finales=[]
    alphas=[]
    for i in range(len(espectro_lineas)):
        if len(espectro_lineas[i])>1:
            if 'SPEC' in espectro_lineas[i][1] and 'NSPEC' not in espectro_lineas[i][1]:
                componente_filas_spec.append(i)  #me dice en que componente esta el spectro de la posicion

            if 'time:' == espectro_lineas[i][1] :
                fase=espectro_lineas[i][2]

                spec.append(float(fase)) #le saque el round porque me daba espectros repetidos cuando, si lo quiero poner deberia quiza usar un drop duplicates en el
                                            #calculo del rms y mad
                fases=spec
                #print(fase)
            else:
                if MJD==True:
                    if len(espectro_lineas[i])==5:
                        if 'MJD' in espectro_lineas[i][4] and 'SPEC' in espectro_lineas[i]:
                            fase=espectro_lineas[i][4]
                            fase=float(fase.strip('MJD='))
                            spec.append(fase) #saque el round
                            fases=spec
            if ot==True:
                if 'ini:' in espectro_lineas[i][1]:
                    inicial=espectro_lineas[i][2]
                    iniciales.append(float('%.3f'%float(inicial)))
                if 'fin:' in espectro_lineas[i][1]:
                    final=espectro_lineas[i][2]
                    finales.append(float('%.3f'%float(final)))
                if 'alpha:' in espectro_lineas[i][1]:
                    alpha=espectro_lineas[i][2]
                    alphas.append(float('%.3f'%float(alpha)))


    ESPECTRO=[]
    for j in range(len(componente_filas_spec)):
        #nombre_spec = str(spec[j])
        #print (nombre_spec)
        espectro1=[]
        if j==len(componente_filas_spec)-1:
            final=len(espectro_lineas)
            lineas_espectro=np.arange(componente_filas_spec[j]+1,final,1)
            #print (lineas_espectro)
            for i in lineas_espectro:
                if espectro_lineas[i][1]=='WAVE':
                    continue
                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])

        else:
            if ot==True:
                res=4 #esto es para que lea los archivos bien ya que los OT tienen info extra
            else:
                res=1
            lineas_espectro=np.arange(componente_filas_spec[j]+1,componente_filas_spec[j+1]-res,1) #me dice las lineas que tengo q guardar por filtro
            #print (lineas_espectro)
            for i in lineas_espectro:

                if len(espectro_lineas[i])==0:
                    break
                if espectro_lineas[i][1]=='WAVE':
                    continue

                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])
        if as_pandas==True:
            #print(len(espectro1[0]))
            #print(espectro1)
            try:
                if len(espectro1[0])==2:
                    espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux"])
                elif len(espectro1[0])==3:
                    espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux",'fluxerr'])
                else:
                    return print('Imposible llevar a pandas, revisa las columnas')
            except:
                espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux"])

        ESPECTRO.append(espectro1)
    #print('Phases:',fases)

    if ot==True:
        return ESPECTRO,fases,iniciales,finales,alphas
    else:
        return ESPECTRO,fases


import csv

def leer_spectra_OT(ruta_archivo):
    with open(ruta_archivo) as archivo:
        lector_csv = csv.reader(archivo, delimiter='\t',skipinitialspace=True)
        ESPECTRO = []
        fases = []
        iniciales = []
        finales = []
        alphas = []
        espectro_actual = []
        for linea in lector_csv:

            if not linea:
                continue
            if linea[0].startswith('#'):
                if 'SPEC' in linea[0] and len(espectro_actual)>0:
                    ESPECTRO.append(pd.DataFrame(espectro_actual,columns=['wave','flux']))
                    espectro_actual = []
                elif 'ini' in linea[0]:
                    iniciales.append(float(linea[0].split(':')[1]))
                elif 'fin' in linea[0]:
                    finales.append(float(linea[0].split(':')[1]))
                elif 'alpha' in linea[0]:
                    alphas.append(float(linea[0].split(':')[1]))
                elif 'time' in linea[0]:
                    fases.append(float(linea[0].split(':')[1]))
            else:
                #print(linea)
                linea=list(filter(None, linea)) #para eliminar cuando empieza con un elemento vacio en el espectro
                #print(linea)
                longitud_onda = float(linea[0])
                flujo = float(linea[1])
                espectro_actual.append((longitud_onda, flujo))

        ESPECTRO.append(pd.DataFrame(espectro_actual,columns=['wave','flux'])) #este es para el ultimo espectro
        return ESPECTRO, fases, iniciales, finales, alphas




In [ ]:
def sin_vs_obs(syntetic_df,V,FILTRO_,plot_fases=[],plot_syn=True,fig_title='',K=False,ot=False,mag_to_flux=False,interactive=False,get_ALR=False,use_cte=True,df_othersyn=[]):
    #el V es un string con el nombre de la banda
    #recive la curva de luz sintetica y la observada

    cteB=6.460803024157998e-09
    cteV=3.67558126207669e-09
    cteR=2.2319872881446082e-09
    cteI=1.1769629663231628e-09
    cteU=4.346809582387867e-09

    cteu=9.512689427618875e-09#859.5e-11
    cteg=4.791168585588861e-09#466.9e-11
    cter=2.818556152283821e-09#278.0e-11
    ctei=1.9069984363571034e-09#185.2e-11
    ctez=1.413365541656362e-09#131.5e-11
    arr_ctes=['cteB','cteV','cteR','cteI','cteU','cteu','cteg','cter','ctei','ctez']
    arr_val_ctes=[cteB,cteV,cteR,cteI,cteU,cteu,cteg,cter,ctei,ctez]

              #poner el indice de el plot en el que esta la curva V

    DJ=[]
    flux=[]
    flux_err=[]
    constante='cte'+V #pongo V pero es cualquier banda
    #print (constante)
    for jj in range(len(arr_ctes)):
            if constante==arr_ctes[jj]: #recorremos el arreglo de constantes en busca de la que le corresponde
                print (constante,arr_ctes[jj],arr_val_ctes[jj])
                mul=arr_val_ctes[jj]

    if use_cte==False:
        mul=1


    #lc_df=pd.DataFrame( FILTRO_) #data frame de la curva real
    lc_df=FILTRO_
    lc_df.rename(columns={0:'mjd',1:'flux',2:'flux_err',
                          3:'Upperlimit',4:'Instrument',5:'Telescope',6:'Source'},inplace=True)
    lc_df['flux_err'].replace(0,0.05,inplace=True)

    if len(lc_df.keys())>3:
        lc_df=lc_df.loc[lc_df.loc[:, 'Upperlimit'] !='T'] #si ES T es upperlimit
    #print('REAL DATA FRAME')
    #print(lc_df)
    DJ=lc_df['mjd']
    if len(DJ)<1:
        return print('No hay datos en la banda:',V)

    if mag_to_flux==True:
        lc_df['flux']=10**(-2*lc_df['flux']/5)
        lc_df['flux_err']=(lc_df['flux_err']*lc_df['flux'])/1.086
    flux=lc_df['flux']*mul
    flux_err=lc_df['flux_err']*mul


    if ot==True:
        syn_df=pd.DataFrame( syntetic_df) #data frame de la curva real
        syn_df.rename(columns={0:'mjd',1:'flux',2:'flux_err',
                              3:'Upperlimit',4:'Instrument',5:'Telescope',6:'Source',7:'Ini',8:'Fin',9:'Alpha'},inplace=True)

    if ot==False:
        syn_df=pd.DataFrame( syntetic_df) #data frame de la curva real
        syn_df.rename(columns={0:'mjd',1:'flux',2:'flux_err',
                              3:'Upperlimit',4:'Instrument',5:'Telescope',6:'Source'},inplace=True)
        if len(df_othersyn)>0:
            df_othersyn=pd.DataFrame( df_othersyn) #data frame de la curva real
            df_othersyn.rename(columns={0:'mjd',1:'flux',2:'flux_err',
                              3:'Upperlimit',4:'Instrument',5:'Telescope',6:'Source'},inplace=True)


    #print('SINTETIC DATA FRAME')
    #print(syn_df)
    #-------------------------------------------------------------------------------------------

    import sys
    #sys.path.append('C:/Users/equipo/Dropbox/ALR/src')
    #sys.path.append('/data1/mauricio/ALR/src')
    #sys.path.append("G:\\Mi unidad\\Work\\ALR\src")
    #from ALR import Automated_Loess_Regression
    #import plot_ALR

    save_interactive='n'
    alpha=0
    nplot=0
    while save_interactive=='n':
        #GP(x,y,yerr,evaluar)
        #return x_pred,y_pred,sigma
        ALR = interpolate.interp1d(DJ, flux)
        if min(DJ)<0 and max(DJ)>0:x_interp=np.arange(int(min(DJ)),int(max(DJ)+1)) # -> el +2 es para que calze con "sampleo"
        if min(DJ)>0 and max(DJ)>0:x_interp=np.arange(int(min(DJ)),int(max(DJ)+1))
        if min(DJ)<0 and max(DJ)<0:x_interp=np.arange(int(min(DJ)+1),int(max(DJ)))
        if min(DJ)>0 and max(DJ)<0: print('Checkea los dias julianos')
        #print(DJ)
        #print(x_interp)
        if len(x_interp)<2:
            return print('No se puede hacer Loess con un Solo dato')
        y_ALR=ALR(x_interp) #este es para el plot
        y_err_ALR=np.sqrt(y_ALR)


        x_df=syn_df.loc[syn_df.loc[:, 'mjd'] >=min(DJ)]
        x_df=x_df.loc[x_df.loc[:, 'mjd'] <=max(DJ)]

        #print(x_df['mjd'])
        if len(x_df)==0:
            return print('No hay espectros dentro del rango de curva de luz')
        y_ALR2=ALR(x_df['mjd'])
        y_err_ALR2=np.zeros(len(y_ALR2))
        #este para calcular K y los residuales
        #print('el loess',y_ALR2)
        #fig=plot_ALR([ALR],invert_y_axis=False)
        #plt.errorbar(x_interp,y_ALR,yerr=y_err_ALR,fmt='-')
        #plt.plot(x_interp,y_ALR,'-')
        #plt.plot(x_interp, y_ALR+3.0*y_err_ALR, '--', color='green'  , lw=1, zorder=1)
        #plt.plot(x_interp, y_ALR-3.0*y_err_ALR, '--', color='green'  , lw=1, zorder=1)
        #plt.show()

        x_interp_df=pd.DataFrame(x_interp,columns=['mjd'])
        y_ALR_df=pd.DataFrame(y_ALR,columns=['flux'])
        y_err_ALR_df=pd.DataFrame(y_err_ALR,columns=['flux_err'])
        x_interp_df.reset_index(drop=True,inplace=True)
        ALR_df=pd.merge(x_interp_df, y_ALR_df, right_index=True, left_index=True)
        ALR_df=pd.merge(ALR_df, y_err_ALR_df, right_index=True, left_index=True)

        x_df=pd.DataFrame(x_df['mjd'],columns=['mjd'])
        y_ALR2_df=pd.DataFrame(y_ALR2,columns=['flux'])
        y_err_ALR2_df=pd.DataFrame(y_err_ALR2,columns=['flux_err'])
        x_df.reset_index(drop=True,inplace=True)
        #print(x_df)
        #print(y_ALR2_df)
        #print(y_err_ALR2_df)
        ALR2_df=pd.merge(x_df, y_ALR2_df, right_index=True, left_index=True)
        ALR2_df=pd.merge(ALR2_df, y_err_ALR2_df, right_index=True, left_index=True)


        #DJ_band_df=pd.DataFrame(np.array(DJ_band),columns=['mjd']) #Estos son los espectros
        #FLUX_band_df=pd.DataFrame(np.array(FLUX_band),columns=['flux'])
        #syn_df=pd.merge(DJ_band_df, FLUX_band_df, right_index=True, left_index=True)
        #print('ALR DATA FRAME')
        #print(ALR2_df)

        #print('MERGE DATA FRAME')
        merge = pd.merge(syn_df, ALR2_df, how='inner', on=['mjd'])

        #print('Syn df:',syn_df)
        #print('Loess df (evualuated on syn):',ALR2_df)
        #print('Merge df:',merge)

        #print(DJ)
        #print(DJ_band)
        #print(merge['mjd'])
        k=merge['flux_x']/merge['flux_y']
        #------------------------------------------------------------------
        if fig_title!='':
            fig = plt.figure(figsize=(11,8))
            fig.subplots_adjust(left=0.12, bottom=0.2, hspace=0.25, right=0.99, top=0.96)
            ax_ = fig.add_subplot(211)

            DJ_band=syn_df['mjd']
            FLUX_band=syn_df['flux']
            if len(df_othersyn)>0:
                DJ_band_othersyn=df_othersyn['mjd']
                FLUX_band_othersyn=df_othersyn['flux']

            #ax_.errorbar(DJ,flux,yerr=flux_err,fmt='o',label='OBS',zorder=0,alpha=0.7)
            if plot_syn==True:
                ax_.plot(DJ_band,FLUX_band,'o',label='Synthetic',zorder=1,alpha=0.7)
                if len(df_othersyn)>0:
                    ax_.plot(DJ_band_othersyn,FLUX_band_othersyn,'o',label='other',zorder=5,alpha=0.7,color='k')

                #ax_.plot(merge['mjd'],merge['flux_x']/k,'^',label='corrected',color='red',zorder=0)
            ax_.plot(x_interp,y_ALR,'-',zorder=2,label='Interpolation')

            ax_.plot(x_interp, y_ALR+y_err_ALR, '--', color='green'  , lw=1, zorder=2)
            ax_.plot(x_interp, y_ALR-y_err_ALR, '--', color='green'  , lw=1, zorder=2)
            plt.title('Filtro '+V)
            plt.xlabel(r'$\mathtt{MJD}$')
            plt.ylabel(r'$\mathtt{Flux}$')
            if fig_title!='': plt.title('Filtro '+V+' '+fig_title)

            ax_.tick_params('both', length=6, width=1, which='major', direction='in')
            ax_.tick_params('both', length=3, width=1, which='minor', direction='in')
            ax_.xaxis.set_ticks_position('both')
            ax_.yaxis.set_ticks_position('both')
            plt.legend()
            plt.xlim(min(x_interp)-10,max(x_interp)+10)
            #plt.rcParams['figure.figsize'] = [10, 8]
            #plt.figure(999)
        #--------------------------------------------------------------------------


        #fig = plt.figure(19085463,figsize=(9,9))
        #fig.subplots_adjust(left=0.12, bottom=0.05, hspace=0.0, right=0.99, top=0.96)

        residuales=(merge['flux_x']-merge['flux_y'])/merge['flux_y'] #sin-ALR

        if fig_title!='':
            ax  = fig.add_subplot(212)
            ax.plot(merge['mjd'],np.zeros(len(merge['mjd'])),'--',ms=1,color='black')
            ax.plot(merge['mjd'],residuales,'o',ms=10,color='red')
            ax.tick_params('both', length=6, width=1, which='major', direction='in')
            ax.tick_params('both', length=3, width=1, which='minor', direction='in')
            ax.xaxis.set_ticks_position('both')
            ax.yaxis.set_ticks_position('both')
            ax.set_ylabel(r'$\mathtt{Residuals = \frac{F_{syn}-F_{ALR}}{F_{ALR}}}$')
            ax.set_xlabel(r'$\mathtt{MJD}$')
            plt.xlim(min(x_interp)-10,max(x_interp)+10)
            #plt.legend()


        if len(plot_fases)>0:
            for w in plot_fases:
                ax_.axvline(w,ls='--',color='blue',alpha=0.5)
                ax.axvline(w,ls='--',color='blue',alpha=0.5)
        plt.show()

        if interactive==True:
            save_interactive=input('Save? y/n')
            if save_interactive=='n':
                alpha=float(input('Input the alpha value: '))
            if save_interactive=='y':
                continue
            while save_interactive!='y' and save_interactive!='n':
                print('Pls use y/n notation.')
                save_interactive=input('Save? y/n')
        if interactive==False:
            save_interactive='y'
    nplot=nplot+1

    if ot==True:
        if fig_title!='' and K==False:
            return residuales,fig,merge['mjd'],merge['Ini'],merge['Fin'],merge['Alpha']
        if K==True and fig_title!='' and get_ALR==False:
            return residuales,fig,merge['mjd'],k,merge['Ini'],merge['Fin'],merge['Alpha']
        if K==True and fig_title!='' and get_ALR==True:
            return residuales,fig,merge['mjd'],k,merge['Ini'],merge['Fin'],merge['Alpha'],ALR_df
        if fig_title=='' and K==True and get_ALR==True:
            return residuales,merge['mjd'],k,merge['Ini'],merge['Fin'],merge['Alpha'],ALR_df
        else:
            return residuales
    else:

        if fig_title!='' and K==False:
            return residuales,fig

        elif K==True and fig_title!='' and get_ALR==False:
            return residuales,fig,merge['mjd'],k

        elif K==True and fig_title!='' and get_ALR==True:
            return residuales,fig,merge['mjd'],k,ALR_df #este estoy entrando
        elif K==True and fig_title=='' and get_ALR==True:
            return residuales,merge['mjd'],k,ALR_df #este estoy entrando
        else:
            print('hola')
            return residuales


In [ ]:
def unir_filtros(FILTRO,filtros,jc_sloan=True):
    filtro=filtros
    FILTRO3=[]
    filtros_convi2=[]
    repetido=[]
    filtros3=[]

    if jc_sloan==True:

            for j in range(20): #le pongo 20 para que revise 20 vees y saque todo
                for i in filtro:
                    if i!='B' and i!='V' and i!='R' and i!='I' and i!='U' and  i!='i' and i!='u'  and i!='g' and i!='r' and i!='z' and i!="u'" and i!="g'" and i!="r'" and i!="i'" and i!="z'" :
                        remove_filter(i,FILTRO,filtro)


    for i in range(len(filtro)):
        #print(filtro[i])

        componentes=[]
        componentes.append(i)
        if filtro[i]  in repetido:
            continue

        for j in range(len(filtro)):
                if filtro[i][:4]=='usno':
                    if filtro[i]!=filtro[j] and filtro[i][5]== filtro[j][0] and filtro[j][:4]!='usno':
                        componentes.append(j)
                        repetido.append(filtro[j])
                if filtro[i]!=filtro[j] and filtro[i][0]==filtro[j][0] and filtro[i][:4]!='usno' :
                    componentes.append(j)
                    repetido.append(filtro[j])
        componentes=sorted(componentes)
        FILTRO2=[]
        for k in componentes:

            FILTRO2=FILTRO[k]+FILTRO2

        FILTRO3.append(sorted(FILTRO2))
        filtros_convinados=[]
        for qq in componentes:
            filtros_convinados.append(filtro[qq])
        filtros_convi2.append(filtros_convinados)


        for jj in range(len(FILTRO2)):
            DJ2=[]
            flux2=[]
            fluxerr2=[]
            for ii in range(len(FILTRO2)):

                DJ2.append(np.float(FILTRO2[ii][0]))
                flux2.append(np.float(FILTRO2[ii][1]))

            #necesitamos las barras de error
                fluxerr2.append(np.float(FILTRO2[ii][2]))
            xerr2=np.zeros(len(flux2)) #porque no hay error en X

        fig=plt.figure(jj)
        plt.errorbar(DJ2, flux2, fluxerr2,xerr2,'.','g')
        plt.grid()
        plt.xlabel('Dia Juliano')
        plt.ylabel('FLUJO')
        plt.gca().invert_yaxis()


        plt.title('Curva de Luz filtro '+str(filtros_convinados))
        plt.show()
        #repetido.append(filtro[i])


    for i in range(len(filtros_convi2)):
        filtros3.append(filtros_convi2[i][0][0][0])


    return FILTRO3,filtros3,filtros_convi2

In [ ]:
def Loess_fit(FILTRO_,V,mag_to_flux=False,interactive=False,fig_title='',use_cte='True',alpha=0,corte=40,plot=True): #entra solo una curva
    cteB=6.460803024157998e-09
    cteV=3.67558126207669e-09
    cteR=2.2319872881446082e-09
    cteI=1.1769629663231628e-09
    cteU=4.346809582387867e-09

    cteu=9.512689427618875e-09#859.5e-11
    cteg=4.791168585588861e-09#466.9e-11
    cter=2.818556152283821e-09#278.0e-11
    ctei=1.9069984363571034e-09#185.2e-11
    ctez=1.413365541656362e-09#131.5e-11
    arr_ctes=['cteB','cteV','cteR','cteI','cteU','cteu','cteg','cter','ctei','ctez']
    arr_val_ctes=[cteB,cteV,cteR,cteI,cteU,cteu,cteg,cter,ctei,ctez]
    constante='cte'+V #pongo V pero es cualquier banda
    print (constante)
    for jj in range(len(arr_ctes)):
            if constante==arr_ctes[jj]: #recorremos el arreglo de constantes en busca de la que le corresponde
                print (constante,arr_ctes[jj],arr_val_ctes[jj])
                mul=arr_val_ctes[jj]
                if use_cte=='False':
                    mul=1




    lc_df=pd.DataFrame( FILTRO_) #data frame de la curva real
    lc_df.rename(columns={0:'mjd',1:'flux',2:'flux_err',
                          3:'Upperlimit',4:'Instrument',5:'Telescope',6:'Source'},inplace=True)
    #lc_df['flux_err'].replace(0,0.05,inplace=True)

    #if len(lc_df.keys())>3:
     #   lc_df=lc_df.loc[lc_df.loc[:, 'Upperlimit'] !='T'] #si ES T es upperlimit, por lo que se elimina del df

    DJ=lc_df['mjd']
    if len(DJ)<5:
        print('No hay suficientes datos (5) datos en la banda para realizar el fit:',V)
        ALR_df=[]
        return ALR_df

    #return lc_df
    if mag_to_flux==True:
        lc_df['flux']=10**(-2*lc_df['flux']/5)
        lc_df['flux_err']=(lc_df['flux_err']*lc_df['flux'])/1.086
        lc_df['flux_err'].replace(0,0.05,inplace=True)
    lc_df['flux_err'].replace(0,np.max(lc_df['flux'])*0.1,inplace=True)
    flux=lc_df['flux']*mul
    flux_err=lc_df['flux_err']*mul

    import sys
    sys.path.append('/home/mauri/Escritorio/ALR/src')
    sys.path.append('/data1/mauricio/ALR/src')
    sys.path.append("G:\\Mi unidad\\Work\\ALR\src")
    from ALR import Automated_Loess_Regression
    import plot_ALR

    save_interactive='n'
    
    while save_interactive=='n':
        if len(alpha)==1:

            ALR = Automated_Loess_Regression(DJ, flux,err_y=flux_err,alpha=alpha[0])
            if min(DJ)<0 and max(DJ)>0:x_interp=np.arange(int(min(DJ)),int(max(DJ)+1)) # -> el +2 es para que calze con "sampleo"
            if min(DJ)>0 and max(DJ)>0:x_interp=np.arange(int(min(DJ)),int(max(DJ)+1))
            if min(DJ)<0 and max(DJ)<0:x_interp=np.arange(int(min(DJ)+1),int(max(DJ)))
            if min(DJ)>0 and max(DJ)<0: print('Checkea los dias julianos')
            #print(DJ)
            #print(x_interp)
            if len(x_interp)<5:
                return print('No se recomienda hacer Loess con menos de 5')
            y_ALR,y_err_ALR=ALR.interp(x_interp) #este es para el plot

            #print(x_df['mjd'])
            #print('el loess',y_ALR2)

            x_interp_df=pd.DataFrame(x_interp,columns=['mjd'])
            y_ALR_df=pd.DataFrame(y_ALR,columns=['flux'])
            y_err_ALR_df=pd.DataFrame(y_err_ALR,columns=['flux_err'])
            x_interp_df.reset_index(drop=True,inplace=True)
            ALR_df=pd.merge(x_interp_df, y_ALR_df, right_index=True, left_index=True)
            ALR_df=pd.merge(ALR_df, y_err_ALR_df, right_index=True, left_index=True)


            y_ALR2,y_err_ALR2=ALR.interp(DJ) #para los residuales
        
        if len(alpha)==2:
            print('ENTRANDO AL DIVIDIDO')

            #--------------
            lc_df1=lc_df[:corte]
            lc_df2=lc_df[corte:]
            #------------
            DJ1=lc_df1['mjd']
            flux1=lc_df1['flux']
            flux_err1=lc_df1['flux_err']
            lc_upper_df1=lc_df1.loc[lc_df1.loc[:, 'Upperlimit'] =='T']
            flux_upper1=lc_upper_df1['flux']
            flux_err_upper1=lc_upper_df1['flux_err']
            if len(DJ1)<5:
                print('No hay suficientes datos (5) datos en la banda para realizar el fit:',V)
                ALR_df=[]
                return ALR_df
                continue
            
            DJ2=lc_df2['mjd']
            flux2=lc_df2['flux']
            flux_err2=lc_df2['flux_err']
            lc_upper_df2=lc_df2.loc[lc_df2.loc[:, 'Upperlimit'] =='T']
            flux_upper2=lc_upper_df2['flux']
            flux_err_upper2=lc_upper_df2['flux_err']
            if len(DJ2)<5:
                print('No hay suficientes datos (5) datos en la banda para realizar el fit:',V)
                ALR_df=[]
                return ALR_df
                continue
            
            #-----------
            ALR1=Automated_Loess_Regression(DJ1, flux1,err_y=flux_err1,alpha=alpha[0])
            ALR2=Automated_Loess_Regression(DJ2, flux2,err_y=flux_err2,alpha=alpha[1])
            
            if min(DJ1)<0 and max(DJ1)>0:x_interp1=np.arange(int(min(DJ1)),int(max(DJ1)+1)) # -> el +2 es para que calze con "sampleo"
            if min(DJ1)>0 and max(DJ1)>0:x_interp1=np.arange(int(min(DJ1)),int(max(DJ1)+1))
            if min(DJ1)<0 and max(DJ1)<0:x_interp1=np.arange(int(min(DJ1)+1),int(max(DJ1)))
            if min(DJ1)>0 and max(DJ1)<0: print('Checkea los dias julianos')
                
            if min(DJ2)<0 and max(DJ2)>0:x_interp2=np.arange(int(min(DJ2)),int(max(DJ2)+1)) # -> el +2 es para que calze con "sampleo"
            if min(DJ2)>0 and max(DJ2)>0:x_interp2=np.arange(int(min(DJ2)),int(max(DJ2)+1))
            if min(DJ2)<0 and max(DJ2)<0:x_interp2=np.arange(int(min(DJ2)+1),int(max(DJ2)))
            if min(DJ2)>0 and max(DJ2)<0: print('Checkea los dias julianos')
            #print('DJ:',DJ)
            #print('flux;',flux)
            #print('x para interpolar:',x_interp)
            if len(x_interp1)<5:
                print('No se recomienda hacer Loess con menos de 5')
                continue
            if len(x_interp2)<5:
                print('No se recomienda hacer Loess con menos de 5')
                continue
            
            y_ALR1,y_err_ALR1=ALR1.interp(x_interp1) #este es para el plot
            y_ALR2,y_err_ALR2=ALR2.interp(x_interp2) #este es para el plot
            y_ALR=np.concatenate((y_ALR1, y_ALR2), axis=None)
            y_err_ALR=np.concatenate((y_err_ALR1, y_err_ALR2), axis=None)


            #print(x_df['mjd'])
            #print('el loess',y_ALR2)
            
            x_interp_df1=pd.DataFrame(x_interp1,columns=['mjd'])
            x_interp_df2=pd.DataFrame(x_interp2,columns=['mjd'])
            x_interp_df=pd.concat([x_interp_df1,x_interp_df2])
            
            y_ALR_df1=pd.DataFrame(y_ALR1,columns=['flux'])
            y_ALR_df2=pd.DataFrame(y_ALR2,columns=['flux'])
            y_ALR_df=pd.concat([y_ALR_df1,y_ALR_df2])

            y_err_ALR_df1=pd.DataFrame(y_err_ALR1,columns=['flux_err'])
            y_err_ALR_df2=pd.DataFrame(y_err_ALR2,columns=['flux_err'])
            y_err_ALR_df=pd.concat([y_err_ALR_df1,y_err_ALR_df2])

            
            x_interp_df.reset_index(drop=True,inplace=True)
            y_ALR_df.reset_index(drop=True,inplace=True)
            y_err_ALR_df.reset_index(drop=True,inplace=True)
            #print(y_ALR_df)
            #print(x_interp_df)
            #print(y_err_ALR_df)
            ALR_df=pd.merge(x_interp_df, y_ALR_df, right_index=True, left_index=True)
            #print(ALR_df)
            ALR_df=pd.merge(ALR_df, y_err_ALR_df, right_index=True, left_index=True)
            
            ALR_df.drop_duplicates(subset=['mjd'],inplace=True)
            #print(ALR_df)

            y_ALR_residuals1,y_err_residuals1=ALR1.interp(DJ1) #para los residuales
            y_ALR_residuals2,y_err_residuals2=ALR2.interp(DJ2[:]) #para los residuales
            
            y_ALR_residuals=np.concatenate((y_ALR_residuals1, y_ALR_residuals2), axis=None)

        #------------------------------------------------------------------
        if plot==True:
            fig = plt.figure(1908546223,figsize=(6,8))
            fig.subplots_adjust(left=0.12, bottom=0.05, hspace=0.1, right=0.99, top=0.96)
            ax_ = fig.add_subplot(211)

            ax_.errorbar(DJ,flux,yerr=flux_err,fmt='o',label='OBS',zorder=1)
            ax_.plot(x_interp_df['mjd'],y_ALR,'-',label='Loess fit',zorder=2)
            ax_.plot(x_interp_df['mjd'], y_ALR+3.0*y_err_ALR, '--', color='green'  , lw=1, zorder=1)
            ax_.plot(x_interp_df['mjd'], y_ALR-3.0*y_err_ALR, '--', color='green'  , lw=1, zorder=1)
            if mag_to_flux==False:
                plt.gca().invert_yaxis()
            plt.title('Filtro '+V)
            plt.xlabel(r'$\mathtt{Día}$')
            plt.ylabel(r'$\mathtt{Flux}$')
            if fig_title!='': plt.title('Filtro '+V+' '+fig_title)

            ax_.tick_params('both', length=6, width=1, which='major', direction='in')
            ax_.tick_params('both', length=3, width=1, which='minor', direction='in')
            ax_.xaxis.set_ticks_position('both')
            ax_.yaxis.set_ticks_position('both')
            plt.legend()
            #plt.xlim(min(DJ_band)-3,max(DJ_band)+3)
            #plt.rcParams['figure.figsize'] = [10, 8]
            #plt.figure(999)
            #--------------------------------------------------------------------------


            #fig = plt.figure(19085463,figsize=(9,9))
            #fig.subplots_adjust(left=0.12, bottom=0.05, hspace=0.0, right=0.99, top=0.96)
            ax  = fig.add_subplot(212)

            if len(alpha)==1:
                residuales=((flux-y_ALR2)/flux) #real-loess
            if len(alpha)==2:
                residuales=((flux-y_ALR_residuals)/flux) #real-loess

                

            ax.plot(DJ,np.zeros(len(DJ)),'--',ms=1,color='black')
            ax.plot(DJ,residuales,'o',ms=10,color='red')
            ax.tick_params('both', length=6, width=1, which='major', direction='in')
            ax.tick_params('both', length=3, width=1, which='minor', direction='in')
            ax.xaxis.set_ticks_position('both')
            ax.yaxis.set_ticks_position('both')
            ax.set_ylabel(r'$\mathtt{Residuals = \frac{F_{syn}-F_{ALR}}{F_{ALR}}}$')
            ax.set_xlabel(r'$\mathtt{MJD}$')
            #plt.xlim(min(DJ_band)-3,max(DJ_band)+3)
            #plt.legend()


            plt.show()

        if len(alpha)==1:
            print ('The alpha value for the ALR is:',ALR.alpha)
        if len(alpha)==2:
            print ('The alpha values for the ALR is:',ALR1.alpha,'and',ALR2.alpha)


        if interactive==True:
            save_interactive=input('Save? y/n')
            if save_interactive=='n':
                alpha=float(input('Input the alpha value: '))
            if save_interactive=='y':
                continue
            while save_interactive!='y' and save_interactive!='n':
                print('Pls use y/n notation.')
                save_interactive=input('Save? y/n')
        if interactive==False:
            save_interactive='y'


    return ALR_df


In [ ]:
def calibrar_spectra(ESPECTRO,pos,banda,fases,FILTRO,filtros,return_k=False,tipo='',use_cte=True,mag_to_flux=True,move_lc=0,system='windows'):

    if system=='windows':
        response_df=pd.read_csv('G:/Mi unidad/Work/Universidad/Phd/Practica2/Splines_eachfilter/spline_'+banda[0]+'.txt',sep='\s+',comment='#',header=None)
    if system=='ubuntu':
        response_df=pd.read_csv('/data1/mauricio/Dropbox/Universidad/Phd/Practica2/Splines_eachfilter/spline_'+banda[0]+'.txt',sep='\s+',comment='#',header=None)

    response_df.columns=['wave','response']


    #plt.plot(response_df['wave'],response_df['response'],'--')
    #plt.show()

    #Ahora calculamos la fotometria sintetica y tambien ploteamos el espectro con el porcentaje de la banda para claridad

    #%run /home/mauri/Escritorio/Universidad/Codes/Funciones.ipynb

    spec1_df=ESPECTRO[pos]
    spec1_df.columns=['wave','flux']
    spec1_df
    spec1_df.sort_values('wave',ignore_index=True,inplace=True)

    #regrillamos la respuesta de la banda response_df a la de spec1_df

    import scipy
    from scipy.interpolate import griddata
    regrid=griddata(response_df['wave'], response_df['response'], spec1_df['wave'], method='linear', rescale=False)
    df_regrid=pd.DataFrame({'wave':spec1_df['wave'],'response':regrid})
    df_regrid = df_regrid.dropna(how='any') #eliminamos los nan


    merge = pd.merge(spec1_df, df_regrid, how='inner', on=['wave'])

    #if porcentaje==1:


    syn=Syntetic_photometry_v2(spec1_df['wave'],spec1_df['flux'],response_df['wave'],response_df['response'])
    print('syn|percentage')
    print(syn)


    #Solo para visualizar como cubre la banda
    plt.plot(df_regrid['wave'],df_regrid['response'],'-')
    plt.plot(spec1_df['wave'],spec1_df['flux']/np.max(spec1_df['flux']))
    plt.title(fases[pos])
    plt.show()
    if float(syn[1])<0.7:
        print('No cubre lo suficiente la banda')
        return spec1_df,syn[1]

    #ahora que tenemos la sintetica vamos a calcular la fotometria obs para el dia del espectro
    ##path_lc='/media/mauri/62882B44882B15D7/Universidad/Phd/Practica2/Photometry_unidas2/'+tipo+'/'+name+'_photometry.dat'
    ##if tipo=='Ia':
    ##    path_lc='/media/mauri/62882B44882B15D7/Universidad/Phd/Practica2/Photometry_loess/Ia/'+name+'_photometry.dat'
    ##if tipo=='II':
    ##    if system=='windows':
    ##        path_lc='G:/Mi unidad/Work/Universidad/Phd/Practica2/Photometry_unidas2/'+tipo+'_loess_unidas/'+name+'_photometry.dat'
    ##    if system=='ubuntu':
    ##        path_lc='/data1/mauricio/Dropbox/Universidad/Phd/Practica2/Photometry_unidas2/'+tipo+'_loess_unidas/'+name+'_photometry.dat'
##
 #   #if Ibc==True:
     #   path_lc='/media/mauri/62882B44882B15D7/Universidad/Phd/Practica2/SNdata_oldpractice/Supernovas_def/Photometry_original/'+tipo+'/'+name+'_photometry.dat'


    #"FILTRO,filtros=data_curvas(path_lc)
    print(filtros)


    index_lc=filtros.index(banda[0])
    syn_df=pd.DataFrame({'mjd':fases[pos]+move_lc,'flux':syn[0]},index=[0],dtype='float64')
    #FILTRO_=pd.DataFrame(FILTRO[index_lc],dtype='float64')
    FILTRO_=FILTRO[index_lc]
    #plt.plot(FILTRO_[0],FILTRO_[1])
    #plt.title('Curva '+ filtros[index_lc])
    #plt.gca().invert_yaxis()
    #plt.show()

    residuales,fig,merge_2,k=sin_vs_obs(syn_df,banda[0],FILTRO_,mag_to_flux=mag_to_flux,K=True,fig_title='Comparacion',
                                       use_cte=use_cte)


    #calculamos con el la sintetica dividiendo por el valor de k
    syn2=Syntetic_photometry_v2(spec1_df['wave'],spec1_df['flux']/float(k),response_df['wave'],response_df['response'])
    syn2_df=pd.DataFrame({'mjd':fases[pos]+move_lc,'flux':syn2[0]},index=[0],dtype='float64')
    print('syn|percentage')
    print(syn2)
    residuales,fig,merge_2,k2=sin_vs_obs(syn2_df,banda[0],FILTRO_,mag_to_flux=mag_to_flux,K=True,fig_title='Comparacion',
                                        use_cte=use_cte)

    #ahora que sabemos que el valor de k funciona, procedemos a correguir el espectro
    spectro_df_corr=ESPECTRO[pos].copy()
    spectro_df_corr['flux']=spectro_df_corr['flux']/float(k)

    plt.plot(spectro_df_corr['wave'],spectro_df_corr['flux'])
    plt.title(fases[pos])
    plt.show()
    if return_k==True:
        return spectro_df_corr,syn[1],k
    return spectro_df_corr,syn[1]#el espectro corregido y el porcentaje

In [ ]:
def prom_spec(lista_ESPEC,porcentajes):
    #recibe la lista con los dataframe de los espectros a promediar
    #retorna el dataframe promediado

    #hacemos una lista con los espectros a promediar:
    #lo mas facil es regrillar a 1 amstrong y ver donde se solapan los espectros
    porcentajes=np.array(porcentajes)
    lista_ESPEC_1A=[]
    for i in range(len(lista_ESPEC)):
        wave=lista_ESPEC[i]['wave']
        flux=lista_ESPEC[i]['flux']
        from scipy import interpolate
        cs = interpolate.interp1d(wave, flux,fill_value = "extrapolate")
        #cs = CubicSpline(WAVE_SPEC, FLUX_SPEC)
        xnew=np.arange(min(wave),max(wave)+1,1)
        diferencia=(xnew[0]-int(xnew[0]))

        xnew=xnew-diferencia
        ynew=cs(xnew)
        df_1A=pd.DataFrame({'wave':xnew,'flux':ynew})
        lista_ESPEC_1A.append(df_1A)

        #plt.plot(xnew,ynew)


    for j in range(len(lista_ESPEC_1A)-1):
        if j==0:
            a=lista_ESPEC_1A[j].merge(lista_ESPEC_1A[j+1],how='outer',on=['wave'])
        else:
            a=a.merge(lista_ESPEC_1A[j+1],how='outer',on=['wave'])

    a['flux_y'].fillna(a['flux_x'], inplace=True)
    a['flux_x'].fillna(a['flux_y'], inplace=True) #donde dicen nan, copias el valor del otro spectro


    a['mean']=a.drop('wave',axis='columns').mean(axis=1)

    if len(porcentajes)==2:
        a['mean_W']=np.average([a['flux_x'],a['flux_y']],axis=0,weights=porcentajes)

    prom=pd.DataFrame({'wave':a['wave'],'flux':a['mean']})

    prom.sort_values('wave',ignore_index=True,inplace=True)


    return prom


In [ ]:
def ordenar_bandas(FILTRO,filtros):
    def orden_value(filtro):
        if filtro=='U':
            return 0
        if filtro=='B':
            return 1
        if filtro=='V':
            return 2
        if filtro=='R':
            return 3
        if filtro=='I':
            return 4
        if filtro=='u':
            return 5
        if filtro=='g':
            return 6
        if filtro=='r':
            return 7
        if filtro=='i':
            return 8
        if filtro=='z':
            return 9

    orden=['U','B','V','R','I','u','g','r','i','z']

    parejas_viejas=[]
    parejas_nuevas=[]

    for i in range(len(filtros)):
           parejas_viejas.append([filtros[i],i])

    #las ordenasmos
    filtros.sort(key=orden_value)

    for i in range(len(filtros)):
           parejas_nuevas.append([filtros[i],i])

    new_orden_FILTRO=list(np.zeros(len(filtros))) #creo lista vacia para rellenar con nuevo orden

    for j in range(len(parejas_viejas)):
        banda=parejas_viejas[j][0]
        old_index=parejas_viejas[j][1]
        for k in range(len(parejas_nuevas)):
            if banda==parejas_nuevas[k][0]:
                new_index=parejas_nuevas[k][1]
                #print('El nuevo indice de ',banda, 'es',new_index,'el viejo es',old_index)
                new_orden_FILTRO[new_index]=FILTRO[old_index]

    return new_orden_FILTRO,filtros

In [ ]:
def maximo_lc(tipo,sn,ubuntu=False):
    if tipo=='II':
        if ubuntu==True:
            maximum_df=pd.read_csv("/mnt/g/Mi unidad/Work/OT_unidos_2/OT_combines_test/maximum_II.txt")

        else:
            maximum_df=pd.read_csv("G:\Mi unidad\Work\OT_unidos_2\OT_combines_test\maximum_II.txt")
        max_=float(maximum_df.loc[maximum_df['name']==sn]['fase'])
        return max_
    if tipo=='Ia':
        if ubuntu==True:
             
            df_max=pd.read_csv("/mnt/g/Mi unidad/Work/OT_unidos_2/OT_combines_test/maximum_Ia.txt")
        else:
            df_max=pd.read_csv("G:\Mi unidad\Work\OT_unidos_2\OT_combines_test\maximum_Ia.txt")
        df_max[df_max.name==sn]

    if tipo=='Ibc' or tipo=='Ib' or tipo=='Ic':
            df_max=pd.read_csv(r"G:\Mi unidad\Work\Universidad\Phd\Practica2\maximum_Ibc.dat")
            #df_max=pd.read_csv( "G:\Mi unidad\Spectral Time Series - Ramirez M\Data\maximum_perband.dat")
            df_max[df_max.name==sn]
    
    band_system=['V','R','r','g','i','I','B','U','u','z'] #We use this order to obtain the day of the maximum
    for elemento in band_system:

        maximum=df_max.loc[(df_max['name']==sn) & (df_max['filter']==elemento )]['mjd_max']
        if len(maximum)>0:
            maximum=float(maximum)
            break
    return maximum

In [ ]:
def redden_spectrum_adjusted(la, spec, Rv, ebmv, norm='n'):
    """
    Version en python de codigo en IDL de G. Pignata 2004

    Aplica corrección de enrojecimiento o desenrojecimiento a un espectro dado,
    basándose en la ley de Cardelli. La función también ofrece la opción de normalizar
    el espectro corregido.

    Parámetros:
    la (array de numpy): Array de longitudes de onda en Ångströms.
    spec (array de numpy): Espectro original que se desea corregir.
    Rv (float): Relación de extinción total a selectiva (R_V).
    ebmv (float): Exceso de color E(B-V). Valores positivos para enrojecimiento,
                  negativos para desenrojecimiento.
    norm (str): 'y' para normalizar el espectro corregido, cualquier otro valor
                para no normalizar.

    Devuelve:
    array de numpy: Espectro corregido.

    Notas:
    - La corrección se aplica solo para longitudes de onda entre 3030 Å y 33000 Å.
    - Utiliza la ley de Cardelli para calcular la extinción.
    """

    # Verifica si hay longitudes de onda menores a 3030 Å, que no son válidas
    if np.min(la) <= 1250:
        raise ValueError("No hay corrección para longitudes de onda menores de 1250")

    # Inicializa abs_flux como un arreglo de unos, del mismo tamaño que spec
    abs_flux = np.ones_like(spec)

    # Corrección en el infrarrojo (IR)
    la_ir_indices = np.where((la < 33000) & (la > 9000))[0]
    if len(la_ir_indices) > 0:
        # Calcula y aplica la corrección IR a las partes relevantes de abs_flux
        la_ir = la[la_ir_indices]
        x_ir = 1.0 / (la_ir / 10000.0)
        a_ir = 0.574 * x_ir ** 1.61
        b_ir = -0.527 * x_ir ** 1.61
        abs_flux[la_ir_indices] = 10 ** (-0.4 * ((a_ir * Rv + b_ir) * ebmv))

    # Corrección óptica
    la_opt_indices = np.where((la <= 9000) & (la > 3000))[0]
    if len(la_opt_indices) > 0:
        # Calcula y aplica la corrección óptica
        la_opt = la[la_opt_indices]
        x_opt = 1.0 / (la_opt / 10000.0)
        y = x_opt - 1.82
        a_opt = 1.0 + 0.17699*y - 0.50447*y**2 - 0.02427*y**3 + 0.72085*y**4 + 0.01979*y**5 - 0.77530*y**6 + 0.32999*y**7
        b_opt = 1.41338*y + 2.28305*y**2 + 1.07233*y**3 - 5.38434*y**4 - 0.62251*y**5 + 5.30260*y**6 - 2.09002*y**7
        abs_flux[la_opt_indices] = 10 ** (-0.4 * ((a_opt * Rv + b_opt) * ebmv))
        
    # Corrección UV
    la_uv_indices = np.where((la <= 3000) & (la >= 1250))[0]
    if len(la_uv_indices) > 0:
        la_uv = la[la_uv_indices]
        x_uv = 1.0 / (la_uv / 10000.0)  # Convertir a micrómetros y luego tomar el inverso

        # Calcula Fa(x) y Fb(x)
        Fa = np.where((x_uv > 5.9) & (x_uv < 8), -0.04473 * (x_uv - 5.9)**2 - 0.009779 * (x_uv - 5.9)**3, 0)
        Fb = np.where((x_uv > 5.9) & (x_uv < 8), 0.2130 * (x_uv - 5.9)**2 + 0.1207 * (x_uv - 5.9)**3, 0)

        # Calcula a(x) y b(x)
        a_uv = 1.752 - 0.316 * x_uv - 0.104 / ((x_uv - 4.67)**2 + 0.341) + Fa
        b_uv = -3.090 + 1.825 * x_uv + 1.206 / ((x_uv - 4.62)**2 + 0.263) + Fb

        # Aplica la corrección UV
        abs_flux[la_uv_indices] = 10 ** (-0.4 * ((a_uv * Rv + b_uv) * ebmv))
        

    # Normaliza abs_flux si se requiere
    if norm == 'y':
        abs_flux /= np.mean(abs_flux) if np.mean(abs_flux) > 0 else -np.mean(abs_flux)

    # Calcula el espectro de salida multiplicando el espectro original por abs_flux
    spec_out = spec * abs_flux
    return spec_out

In [ ]:
def write_spetra(ESPECTRO,fases,path,err=False):
    #ahora escribimos el espectro promediado
    file1=open(path,'w')
    
    for j in range(len(fases)):

        file1.write('# time:\t'+str(fases[j])+'\n')
        file1.write('# SPEC \n')
        file1.write('#      WAVE   FLUX')
        df_spec=ESPECTRO[j]
        for ii in range(len(df_spec)):
            file1.write('\n')
            if err==True:
                file1.write(str(df_spec.iloc[ii]['wave'])+'\t'+str(float(df_spec.iloc[ii]['flux']))+'\t'+str(float(df_spec.iloc[ii]['fluxerr'])))
            else:
                file1.write(str(df_spec.iloc[ii]['wave'])+'\t'+str(float(df_spec.iloc[ii]['flux'])))
        file1.write('\n')
    file1.close()

def escribir_curvas_luz(FILTRO, filtros, file_path, sn_name):
    """
    Escribe un archivo de curva de luz en el formato especificado, utilizando los datos de una lista de DataFrames y filtros.

    :param FILTRO: Lista de DataFrames, donde cada DataFrame contiene las columnas de datos para un filtro.
    :param filtros: Lista de nombres de filtros correspondientes a los DataFrames en FILTRO.
    :param file_path: Ruta del archivo a escribir.
    :param sn_name: Nombre de la supernova que se incluirá en el encabezado.
    """
    with open(file_path, 'w') as file:
        # Escribir encabezado
        file.write("###################### HEADER ######################\n")
        file.write(f"# SNNAME:\t{sn_name}\n")
        file.write("###################### HEADER ######################\n")
        
        # Iterar sobre cada filtro y su DataFrame correspondiente
        for i, filtro_df in enumerate(FILTRO):
            file.write("\n\n")
            file.write(f"# FILTER {filtros[i]}\n")
            file.write("#        MJD  MAG  MAGERR  Upperlimit  Instrument  Telescope  Source")
            for _, row in filtro_df.iterrows():
                file.write(f"\n\t{row[0]:.3f}\t{row[1]:.3f}\t{row[2]:.3e}\tF\tnan\tnan\tnan")

In [ ]:
def compute_lc(ESPECTRO,fases,filtros,name='',path2=None,z=0,do_plot=False,use_cte=True,ubuntu=False):

    cteB=6.460803024157998e-09
    cteV=3.67558126207669e-09
    cteR=2.2319872881446082e-09
    cteI=1.1769629663231628e-09
    cteU=4.346809582387867e-09

    cteu=9.512689427618875e-09#859.5e-11
    cteg=4.791168585588861e-09#466.9e-11
    cter=2.818556152283821e-09#278.0e-11
    ctei=1.9069984363571034e-09#185.2e-11
    ctez=1.413365541656362e-09#131.5e-11
    print("Tienes curvas de luz en los fitros:\n",filtros)
    bandas=[]
    if ubuntu==True:
        lista=os.listdir(r"/mnt/g/Mi unidad/Work/Universidad/Phd/Practica2/Splines_eachfilter_2")
    else:
        lista=os.listdir(r"G:\Mi unidad\Work\Universidad\Phd\Practica2\Splines_eachfilter_2")
    nombres_archivos_response= {
    "U": "spline_U.txt",
    "B": "spline_B.txt",
    "V": "spline_V.txt",   
    "R": "bessell_R_ph_lines.dat",#We use this because have the telluric lines
    "I": "bessell_I_ph_lines.dat",#We use this because have the telluric lines
    "u": "spline_u'.txt",
    "g": "spline_g'.txt",
    "r": "spline_r'.txt", #this already have the telluric line
    "i": "spline_i'.txt", #this already have the telluric lines
    "z": "spline_z'.txt", 
    }
    bandas_con_curvas=[]
    for filtro in filtros:
        bandas_con_curvas.append(nombres_archivos_response[filtro])

    print('We are going to comptue syn photometry on bands:\n')

    print(bandas_con_curvas)
    

    ######## guardamos los datos de las bandas con curvas en  datos_bandasspline  

    datos_bandasspline=[] #los spline que ocuparemos que tienen curvas


    response_list=[]
    for i in range(len(bandas_con_curvas)):
        #with open (os.path.join(os.getcwd(),"Data","Splines_eachfilter",str(bandas_con_curvas[i])),'r') as f:     
        if ubuntu==True:
            file_path=os.path.join(r"/mnt/g/Mi unidad/Work/Universidad/Phd/Practica2","Splines_eachfilter_2",str(bandas_con_curvas[i]))
        else:
            file_path=os.path.join(r"G:Mi unidad\Work\Universidad\Phd\Practica2","Splines_eachfilter_2",str(bandas_con_curvas[i]))
        df_response=pd.read_csv(file_path,sep='\s+',comment='#',header=None)
        df_response.columns=['wave','response']
        #plt.plot(df_response.wave,df_response.response)
        #plt.show()
        response_list.append(df_response)
    
    
    datos_espectros_spline=ESPECTRO
    
    SINTETICA=[]

    if path2 is not None:
        # Create output directories
        try:    
            os.mkdir(path2)
        except FileExistsError:
            pass
        
        ##############write####################################################
        file=open(os.path.join(path2,name+"_synphotometry.dat"),"w")
        file.write("###################### HEADER ######################\n")
        file.write("# SNNAME:\t"+ name+"\n")
        file.write("###################### HEADER ######################\n")

    LC_list=[]
    
    for q in range(len(response_list)):

            response_df=response_list[q]


            cont=0
            le=[]
            la=[]
            for i in range(len(ESPECTRO)):
                
                spec1_df=ESPECTRO[i]
                spec1_df=spec1_df.clip(0) #para que no hayan valores negativos
                #spec1_df.columns=["wave","flux"]
                #spec1_df
                spec1_df.sort_values("wave",ignore_index=True,inplace=True)


                #plt.figure(i)    
                #plt.plot(response_df["wave"],response_df["response"],"-")
                #plt.plot(spec1_df["wave_z"],spec1_df["flux"]/np.max(spec1_df["flux"]))
                #plt.title("SPECTRO Y RESPUESTA")
                #plt.show()
                    
                #spec1_df['photons']=convert_to_photons(np.array(spec1_df["flux_z"]), np.arraY(spec1_df["wave_z"]))
                #print(spec1_df)
                syn,porcentaje=Syntetic_photometry_v2(spec1_df["wave"],spec1_df["flux"],response_df["wave"],response_df["response"])
                #print(syn)
                #print('el porcentaje es', porcentaje)

                #merge = pd.merge(spec1_df, response_df, how="inner", on=["wave"])
                #print(merge)
                
                
                #pasamos a mag
                arr_ctes=['cteB','cteV','cteR','cteI','cteU','cteu','cteg','cter','ctei','ctez']
                arr_val_ctes=[cteB,cteV,cteR,cteI,cteU,cteu,cteg,cter,ctei,ctez]
                V=filtros[q]
                constante='cte'+V #pongo V pero es cualquier banda
                #print (constante)
                for jj in range(len(arr_ctes)): 
                    if constante==arr_ctes[jj]: #recorremos el arreglo de constantes en busca de la que le corresponde
                        #print (constante,arr_ctes[jj],arr_val_ctes[jj])
                        mul=arr_val_ctes[jj]
                if use_cte==False:
                    mul=1
                mag=-2.5*np.log10(syn/mul)
                #syn=syn#/mul
                syn=mag
                #porcentaje=len(merge["wave"])/len(response_df)
                
                
                #print("ESTE ES EL SYN Y EL PORCENTAJE",syn,porcentaje)
                if porcentaje*100>=95:
                    if path2 is not None:
                        if cont==0:
                            file.write("\n")  
                            file.write("# FILTER "+str(filtros[q])+"\n")
                            file.write("#        MJD  MAG  MAGERR  Upperlimit  Instrument  Telescope  Source\n")
                        
                        
                    #syn=Syntetic_photometry_same_length(merge["wave"],merge["flux"],merge["wave"],merge["response"])
                    #print("el porcentaje de la banda:",porcentaje)
                    #plt.figure(1000000)
                    la.append(fases[i])
                    le.append(syn)
                ##plt.plot(fases[i],syn,"o")
                #print ("EL PORCENTAJE DE LA BANDA "+str(bandas_con_curvas[q]) +" ES DE :",porcentaje*100)
                #print("Syn:",syn)
                #print("la fase es:",fases[i])
                #print("---------------------------------------------------------------------------------------")

                    cont=1
                    if path2 is not None:
                        file.write("\t"+str("%.3f"%fases[i])+"\t"+str("%.3e"%((syn)))+"\t"+ 
                                                    str(0)+"\t"+str("F")+"\t"+
                                                    "nan"+"\t"+"nan"+"\t"+"nan"+
                                                        "\n")


                else:
                    #print ("EL PORCENTAJE DE LA BANDA "+str(bandas_con_curvas[q]) +" ES DE :",porcentaje*100)
                    #print("NO CUMPLE EL MINIMO")
                    #print("---------------------------------------------------------------------------------------")

                    continue


           
            LC_df=pd.DataFrame({'mjd':np.array(la),'flux':np.array(le),'flux_err':np.zeros(len(la)),3:['F']*len(la),'band':filtros[q]})
            
            if do_plot==True:
                if len(le)!=0:
                    fig = plt.figure(figsize=(7,5))
                    ax= fig.add_subplot(111)
                    ax.plot(la,le,".") #le es la flujo, solo le puse ese nombre porque se me acaban los nombres
                    plt.title(name+" "+filtros[q])
                    #ax.plot(la,new_fluxes,'x',color='black')
                    plt.gca().invert_yaxis()
                    plt.show()
            #pdf.savefig(fig,bbox_inches='tight')
            if len(LC_df)!=0:
                LC_list.append(LC_df)
    

    return LC_list      
            #df_projected=field_projection(np.array(la),new_fluxes,df_obslog,tipo,selected_filter,selected_field,offset,name)


In [ ]:
def correct_redeening(sn,ESPECTRO,fases,ebmv=None,mu=None,z=None,path_save='',reverse=False,to_abs_mag=False,write=False,use_DL=False,ubuntu=False):
    '''
    El codigo busca el archivo modulus donde esta toda la info de las SN
    sn_list: lista de las SN con la extension para leerlas, ex ['SN2005cs.dat','SN2013ej.dat']
    names: Lista con los nombres de la SN sin la extencion, para buscar el nombre en archivo modulus
    path_save: path donde se guarda el nuevo espectro, solos si write ==True
    reverse: Si es False, corrigue por reddening, si es True, Agrega el reddening (proceso inverso)
    to_abs_mag= Si es True, lleva todo a magnitud absoluta o corrigue por ella

    '''

    if ubuntu==True:
        G_path='/mnt/g/Mi unidad'
    else:
        G_path="G:\Mi unidad"
    modulus_path=os.path.join(G_path,'Work','Universidad','Phd','DATA_OSC','all_data',' .cvs')
    modulus=pd.read_csv(modulus_path)

    print(sn)
    name=sn
    NEW_ESPECTRO=[]
    if mu ==None:
        mu=float(modulus[modulus['Sn']==name]['modulus'])
    else:
        mu=mu
    if ebmv==None:
        ebmv=float(modulus[modulus['Sn']==name]['ebmv'])
        
        
        #if ebmv
        #random.choice(distribution)
        
    else:
        ebmv=ebmv
    if z!=None:
        z=z
    else:
        z=float(modulus[modulus['Sn']==name]['z'])
    print("E(B-v),Z,mu")
    print(ebmv,z,mu)
    
    for i in range(len(ESPECTRO)):
        
        df=pd.DataFrame({'wave':ESPECTRO[i].wave,'flux':ESPECTRO[i].flux})
        
        df = df.reset_index(drop=True)
        #print(i,df)
        
        
        if reverse==False:
            #--------correguimos por redshift, esto le saca el redshift original--------.
            df['wave']=df['wave']/(1+z)
    
            #-------------------correguimos por reddenig de la MW-----------
        
            new_spectro=redden_spectrum_adjusted(ESPECTRO[i].wave,ESPECTRO[i].flux,Rv=3.1,ebmv=ebmv)
        
            df['flux']=new_spectro

            #llevamos el flujo a una distancia comun de 10pc para posteriores usos. la formula F10=(d[pc]/10[pc])^2 * Flujo_actual
            #primero sacamos la distancia en parsec con el modulo de distancia--------------
            if to_abs_mag==True:
                d_pc=10**((mu+5)/5) #distancia en parsec

                df['flux']=df['flux']*((d_pc/10)**2)

            #regrillamos
            xnew=np.arange(int(min(df.wave)),max(df.wave)+1,1)
            interpolated_flux = interpolate.interp1d(df.wave,df.flux,fill_value = "extrapolate")
            
            new_df = pd.DataFrame({'wave': xnew,'flux':interpolated_flux(xnew)})



        if reverse==True: #aplicamos todos los procesos de manera inversa.
            if to_abs_mag==True and use_DL==False:
                d_pc=10**((mu+5)/5) #distancia en parsec
                df['flux']=df['flux']/((d_pc/10)**2) #correguimos la magnitud absoluta
            if to_abs_mag==False and use_DL==True:
                DL=DL_calculator(z)
                
                if z!=0:
                    df['wave']=df['wave']*(1+z)
                    df['flux']=df['flux'] * ((1e-5 /DL)**2) #el 1e-5 son 10pc, los pasamos a Mpc para que tenga las mimas unidades de DL, Fobs= Fem*(1e-5/DL)^2
            if to_abs_mag==True and use_DL==True:
                print('No puedes usar to_abs_mag=True y use_DL True al mismo tiempo\nporfavor decide si correguir por mag abs (modulo de distancia) o llevar a un nuevo redshift y calcular DL')
                return None,None
            new_spectro=redden_spectrum_adjusted(df.wave,df.flux,Rv=3.1,ebmv=-ebmv)
            df['flux']=new_spectro
            if use_DL==False:
                df['wave']=df['wave']*(1+z)
            

            #regrillamos
            #print(int(min(df.wave)),max(df.wave)+1)
            xnew=np.arange(int(min(df.wave)),max(df.wave)+1,1)
            interpolated_flux = interpolate.interp1d(df.wave,df.flux,fill_value = "extrapolate")
            
            new_df = pd.DataFrame({'wave': xnew,'flux':interpolated_flux(xnew)})

        
        
        NEW_ESPECTRO.append(new_df)
    
    if write==True:
        write_spetra(NEW_ESPECTRO,fases,os.path.join(path_save,sn))

            

    
    return NEW_ESPECTRO,fases

In [ ]:
dic_constantes={
 'cteB': 6.460803024157998e-09,
 'cteV': 3.67558126207669e-09,
 'cteR': 2.2319872881446082e-09,
 'cteI': 1.1769629663231628e-09,
 'cteU': 4.346809582387867e-09,
 'cteu': 9.512689427618875e-09,
 'cteg': 4.791168585588861e-09,
 'cter': 2.818556152283821e-09,
 'ctei': 1.9069984363571034e-09,
 'ctez': 1.413365541656362e-09
}

In [ ]:
def mangle_iteration_function(spec_df,fase_float,ratios_df,plot=False,return_fig=False):

    min_scale=0.1#min_scale=5*10**-3
    optimization=False#optimization=True

    ext_spec=spec_df
    ratios_df=ratios_df
    wls_eff = np.array(ratios_df.longitud_efectiva)
    used_filters = np.array(ratios_df.banda)
    fase_float=fase_float
    ratios_0=ratios_df.ratio
    # 1st iteration        


    mang_mask, mang_mask_err = GP_interpolation_mangle(spec_df,ratios_df,optimization=True)

    mangled_spec = ext_spec['flux'] / mang_mask
    mangled_spec_err = ((ext_spec['flux'] * mang_mask_err)**2)**0.5

    smangled_spec = {0 : np.array([a for a in zip(ext_spec['wave'],
                                                        mangled_spec, mangled_spec_err)],
                                    dtype=[('wave', '<f8'), ('flux', '<f8'), ('fluxerr', '<f8')])}
    smangled_spec=pd.DataFrame(smangled_spec[0]) #lo convierto a df
    mangling_mask = {0 : (mang_mask, mang_mask_err) }


    #ahora que tengo la funcion de mangleo y mi espectro mangleado, calculare de nuevo la fotometria sintetica
    #para poder sacar de nuevo los ratios
    mangled_phot_list=[]

    #for filt in used_filters:
        #aqui al parecer calcula de nuevo la fotometria pero en el espectro mangleado
    syn_phot_list=compute_lc([smangled_spec],[fase_float],used_filters,name)
    syn_phot_df=pd.concat([df for df in syn_phot_list], ignore_index=True)


    z_p_list=[]
    for banda in syn_phot_df['band']:
        z_p_str='cte'+banda
        z_p_value=dic_constantes[z_p_str]
        #print(banda,z_p_value)
        z_p_list.append(z_p_value)
    z_p_list=np.array(z_p_list)

    #for filt in used_filters:
    #    mangled_phot_list.append(band_flux(filt, use_what=0)[1])
    #magled_photometry_dict = {0 : {'eff_wls': wls_eff, 
    #                                        'fitted_phot': mangled_phot_list,\
    #                                    'used_filters': used_filters}}


    fig=plt.figure(1, figsize=(14,6))
    plt.rc('font', family='serif')
    plt.rc('xtick', labelsize=13)
    plt.rc('ytick', labelsize=13)
    ax1 = plt.subplot2grid((5,1), (0,0), rowspan=2)
    ratios_err=ratios_df.ratio*0.01
    for f,w,r,rerr in zip(used_filters, wls_eff,\
                            ratios_df.ratio, ratios_err):

        flabel = f
        ax1.errorbar(w, r, yerr=rerr, marker="^", ms=8,
                        mfc= 'red', mec='green', linestyle='None',\
                    ecolor= 'black', label='%s'%flabel)
    ax1.errorbar(smangled_spec['wave'], mang_mask, color='green',label='Mangling\nfunction')
    ax1.fill_between(smangled_spec['wave'], mang_mask-mang_mask_err, mang_mask+mang_mask_err, color='green',alpha=0.3)
    ax1.set_ylabel('Synthetic Flux/\nPhotometric Flux', fontsize=13)
    firts_mangled=mang_mask.copy()
    fase_float_title=fase_float-float(maximo_lc(tipo,name))
    ax1.set_title(' %s'%name+'  '+str(fase_float_title) ,fontsize=15)

    #ax1.set_ylim(min(ratios)*(0.9), max(ratios)*(1.1))
    #ax1.set_xlim(1600., 10300.)
    plt.tick_params(axis='x', labelbottom=False)
    plt.legend()



    #calculamos los ratios actuales
    ratios_it_df=compute_ratios(smangled_spec,fase_float,used_filters,all_FILTROS_real)

    ax2 = plt.subplot2grid((5,1), (2,0), rowspan=3)

    ax2.errorbar(smangled_spec['wave'], smangled_spec['flux'],\
                    lw=0.9, color='green', label='Mangled Spectrum')
    #mask_neg = mangled_spec['flux']<0.
    #ax2.errorbar(mangled_spec['wave'][mask_neg], mangled_spec['flux'][mask_neg],\
    #                lw=3.9, color='green', label='Mangled Spectrum 2')

    ax2.errorbar(ext_spec['wave'], ext_spec['flux'],\
                    lw=0.6, color='blue', alpha=1, label=' Spectrum')

    ax2.errorbar(ratios_df.longitud_efectiva,ratios_df.syn_phot*z_p_list,\
                    marker='^', mfc='yellow', mec='black', linestyle='None', ms=9,
                    label='Flux before mangled spec')
    ax2.errorbar(ratios_df.longitud_efectiva, ratios_it_df.syn_phot*z_p_list,\
                    marker='^', mfc='red', mec='black', linestyle='None', ms=9,
                    label='Flux from mangled spec')

    ax2.errorbar(ratios_df.longitud_efectiva,ratios_df.real_phot*z_p_list,\
                    marker='^', mfc='blue', mec='black', linestyle='None', ms=9,
                    label='Photometric flux')
    '''
    ax2.errorbar(np.concatenate([fitted_phot_dict['eff_wls'],outwls_filters_wls_eff]), 
                    np.concatenate([fitted_phot_dict['fitted_phot'], outwls_fitted_phot_list]),\
                    yerr = np.concatenate([fitted_phot_dict['fitted_phot_err'], 
                                        outwls_fitted_photerr_list]),
                    marker='o', mfc='blue', mec='grey',ms=9, 
                    ecolor='blue', linestyle='None', label='Photometric flux')
            
    ax2.fill_between(mangled_spec['wave'], mangled_spec['flux']-mangled_spec['fluxerr'],\
                        mangled_spec['flux']+mangled_spec['fluxerr'], color='green',alpha=0.3)

    '''

    ax2.set_ylabel(r'Flux ($\mathrm{erg}$ $\mathrm{s^{-1} cm^{-2}} \mathrm{\AA} $)', fontsize=13)
    ax2.set_xlabel(r'Wavelength ($\mathrm{\AA}$)', fontsize=13)
    #ax2.set_xlim(1600., 10300.)
    ax2.legend(ncol=1, fontsize=9, loc='upper right')
    plt.subplots_adjust(hspace=0.3, wspace=0.1)
    #fig.savefig(mangledspec_path+'%.2f_mangled_spec.pdf'%spec_number, bbox_inches='tight')
    plt.show()






    # More iterations
    ratios = ratios_0
    max_iteration = 5
    for i in range(max_iteration):
        if max(np.abs(ratios-1.))>0.01:
            #print (i)

            mang_mask, mang_mask_err = GP_interpolation_mangle(smangled_spec,ratios_it_df,optimization=True)
            #print ('iteration', i,ratios_iter)
            smangled_spec['flux'] = smangled_spec['flux'] / mang_mask
            #mangled_spec_err = ( (self.mangled_spec[str(i+1)]['flux'] * mang_mask_err)**2 +\
            #            (self.mangled_spec[str(i+1)]['fluxerr'] * mang_mask)**2 )**0.5
            smangled_spec['fluxerr'] = (smangled_spec['fluxerr'] * mang_mask_err)

            #print ('save new', (i+1))
            #esto solo lo convierte a diccionario
            #self.mangled_spec[i+1] = np.array([a for a in zip(self.ext_spec['wls'], 
            #                                                        mangled_spec, mangled_spec_err)],
            #                            dtype=[('wls', '<f8'), ('flux', '<f8'), ('fluxerr', '<f8')])
            
            mangling_mask[i+1] = (mang_mask, mang_mask_err)
            mangled_phot_list=[]
            #calculamos la fotometria syn del espectro mangleado
            ratios_it_df=compute_ratios(smangled_spec,fase_float,used_filters,all_FILTROS_real)


            ratios = np.copy(ratios_it_df.ratio)
            ratios_err = np.array(ratios_it_df.ratio)*0.01

            
        

            #ploteamos
            
            if plot==True or i==4:
                fig=plt.figure(2, figsize=(14,6))
                plt.rc('font', family='serif')
                plt.rc('xtick', labelsize=13)
                plt.rc('ytick', labelsize=13)
                ax1 = plt.subplot2grid((5,1), (0,0), rowspan=2)
                for f,w,r,rerr in zip(used_filters, wls_eff,\
                                        ratios, ratios_err):

                    flabel = f
                    ax1.errorbar(w, r, yerr=rerr, marker="^", ms=8,
                                    mfc= 'red', mec='green', linestyle='None',\
                                ecolor= 'black', label='%s'%flabel)
                ax1.errorbar(smangled_spec['wave'], firts_mangled, color='gray',label='First Mangling\nfunction')

                ax1.errorbar(smangled_spec['wave'], mang_mask, color='green',label='Mangling\nfunction')
                ax1.fill_between(smangled_spec['wave'], mang_mask-mang_mask_err, mang_mask+mang_mask_err, color='green',alpha=0.3)
                ax1.set_ylabel('Synthetic Flux/\nPhotometric Flux', fontsize=13)

                ax1.set_title('Last iteration'+' %s'%name+'  '+str(fase_float_title) ,fontsize=15)

                #ax1.set_ylim(min(ratios)*(0.9), max(ratios)*(1.1))
                #ax1.set_xlim(1600., 10300.)
                plt.tick_params(axis='x', labelbottom=False)
                ax1.legend(fontsize=7)

                ax2 = plt.subplot2grid((5,1), (2,0), rowspan=3)

                ax2.errorbar(smangled_spec['wave'], smangled_spec['flux'],\
                                lw=0.9, color='green', label='Mangled Spectrum')
                #mask_neg = mangled_spec['flux']<0.
                #ax2.errorbar(mangled_spec['wave'][mask_neg], mangled_spec['flux'][mask_neg],\
                #                lw=3.9, color='green', label='Mangled Spectrum 2')

                ax2.errorbar(ext_spec['wave'], ext_spec['flux'],\
                                lw=0.6, color='blue', alpha=1, label=' Spectrum')
                
                ax2.errorbar(ratios_df.longitud_efectiva,ratios_df.syn_phot*z_p_list,\
                        marker='^', mfc='yellow', mec='black', linestyle='None', ms=9,
                        label='Flux before mangled spec')
                
                ax2.errorbar(ratios_df.longitud_efectiva, ratios_it_df.syn_phot*z_p_list,\
                        marker='^', mfc='red', mec='black', linestyle='None', ms=9,
                        label='Flux from mangled spec')
                
                ax2.errorbar(ratios_df.longitud_efectiva,ratios_df.real_phot*z_p_list,\
                        marker='^', mfc='blue', mec='black', linestyle='None', ms=9,
                        label='Photometric flux')
                plt.legend(fontsize=7)
                plt.show()

                #print ('last_iter', last_iter)
                #final_mangled_spec = smangled_spec
                #spec_number = self.phot4mangling.spec_mjd.values[0]#self.phot4mangling.index.values[0]

        
        else:
            pass

        smangled_spec
        #self.save_mangled_spectrum()
    if return_fig==True:
        return smangled_spec,fig 
    return smangled_spec

In [ ]:
def GP_interpolation_mangle(spec_df,ratios_df,optimization=True):
    import scipy.optimize as opt
    import george
    from george.kernels import ExpSquaredKernel, ConstantKernel, Matern32Kernel


    ext_spec=spec_df
    wls_eff=ratios_df.longitud_efectiva
    ratios=ratios_df.ratio
    ratios_err=ratios*0.01
    optimization=True


    if len(ext_spec['wave'])>10**4:
                # GP struggle to handle such a big number of points
                int_fraction = int(len(ext_spec['wave'])/5000.)
                print ('This spectrum has a huge amount of data points(%i), Im chopping a %i th of them'%(len(ext_spec['wave']), int_fraction))
                full_wls = ext_spec['wave'][::int_fraction]
    else:
        full_wls = ext_spec['wave']


    norm_wls = np.median(full_wls)
    full_wls_normed = full_wls/norm_wls
    wls_eff_normed = np.array(wls_eff)/norm_wls
        
    offset=1.
    norm = np.mean(ratios)
    ratios_normed = np.array(ratios)/norm-offset
    ratios_err_normed = np.array(ratios_err)/norm

    if len(ratios_normed)<1:
        print (np.ones(len(full_wls_normed))*np.nan, np.ones(len(full_wls_normed))*np.nan)
    else:
        def ll(p):
            #print (np.exp(p))
            if (np.exp(p)[1]<5*10**-3):#|(np.exp(p)[1]>10**5):
                return np.inf
            else:
                gp.set_parameter_vector(p)
                return -gp.lnlikelihood(ratios_normed, quiet=False)#
        def grad_ll(p):
            gp.set_parameter_vector(p)
            return -gp.grad_lnlikelihood(ratios_normed, quiet=False)

        def ll(p):
            gp.set_parameter_vector(p)
            scale = np.exp(gp.get_parameter_dict()['kernel:k2:metric:log_M_0_0'])
            if scale<0.09:
                return np.inf
            else:
                return -gp.lnlikelihood(ratios_normed, quiet=False)#
        
        k= np.var(ratios_normed)*Matern32Kernel(0.3)
        wls_eff_normedT = np.atleast_2d(wls_eff_normed).T
        #print(wls_eff_normedT)
        gp = george.GP(k)
        gp.compute(wls_eff_normedT, (ratios_err_normed))
        if optimization:
            try:
                p0=gp.get_parameter_vector()
                results = opt.minimize(ll, p0, jac=grad_ll)
                print ('SCALE:',  '%.4f'%np.exp(results.x[1]))
            except:
                pass
                print ('*** GP optimization failed ***'*10)
        #print (results)
        gp.set_parameter_vector(results.x)
        mu, cov = gp.predict(ratios_normed, full_wls_normed)
        std = np.sqrt(np.diag(cov))
            
        if len(ext_spec['wave'])>10**4:
            # GP struggle to handle such a big number of points
            mu_full = np.interp(ext_spec['wave'], ext_spec['wave'][::int_fraction], mu)
            std_full = np.interp(ext_spec['wave'], ext_spec['wave'][::int_fraction], std)
        else:
            mu_full = mu
            std_full = std

    
    #retorna la funcion de mangleo y el error asocaido
    return norm*(mu_full+offset), np.abs(norm*(std_full))

In [ ]:
def compute_ratios(spec_df,fase_float,filtros_real,all_FILTROS_real,delete_filter=[]):
       '''
       spec_ : spectro como dataframe, wave,flux
       fase_: float de la fase
       filtros_real: filtros de la fotometria real que esta en FILTROS_REAL
       all_FILTROS_real:  Dataframe con todas las curvas de luces en todas las bandas

       resumen: el codigo calcula la fotometria sintetica del espectro para todas las bandas en la lista filtros,
       despues itera sobre todas las banas de filtros real para calcular los ratios

       return: retorna un df con los ratios por banda y la longitud efectiva
       '''
       #para eso usamos la funcion compute_lighcurve, calculamos sobre todos los filtros que hay en real
       #syn_phto aqu es una lista, con los dataframes de la banda que les corresponde
       syn_phot_list=compute_lc([spec_df],[fase_float],filtros_real,name)
       syn_phot_df=pd.concat([df for df in syn_phot_list], ignore_index=True) #pasamos a pandas porq es mas facil de manejar

       #ahora que tengo la sintetica para todos los filtros, calculo el ratio
       ratios_data = {'banda': [], 'real_phot': [],'syn_phot': [],'ratio': [], 'longitud_efectiva': []}

       for band in filtros_real:
              try:
                     real_phot=float(all_FILTROS_real.loc[(all_FILTROS_real['mjd']==fase_float) & (all_FILTROS_real['band']==band) ].flux)
              except: #por si no hay fotometria real no se puede calcualr K
                     continue
              syn_phot=syn_phot_df.loc[syn_phot_df['band']==band].flux #la phot de una badna especifica para calcular ratio
              if len(syn_phot)==0:
                     continue
              else:
                     syn_phot=float(syn_phot)

              #ahora que tenemos ambas calculamos el ratio
              #pasamos a flujo para hacer el cociente
              syn_phot=10**(-0.4*syn_phot)
              real_phot=10**(-0.4*real_phot)
              ratio=syn_phot/real_phot

              
              longitudes_efectivas={
              'U': 3600.0,
              'B': 4380.0,
              'V': 5450.0,
              'R': 6410.0,
              'I': 7980.0,
              'u': 3560.0,
              'g': 4830.0,
              'r': 6260.0,
              'i': 7670.0,
              'z': 9100.0
              }

              # Añadiendo los datos al diccionario
              ratios_data['banda'].append(band)
              ratios_data['real_phot'].append(real_phot)
              ratios_data['syn_phot'].append(syn_phot)
              ratios_data['ratio'].append(ratio)
              ratios_data['longitud_efectiva'].append(longitudes_efectivas[band])



       df_ratios = pd.DataFrame(ratios_data)
       df_filtrado = df_ratios[~df_ratios['banda'].isin(delete_filter)]

       df_filtrado=df_filtrado.sort_values(by='longitud_efectiva').reset_index(drop=True)

       return df_filtrado

In [ ]:
from math import *
def DL_calculator(z,H0=69.6,WM=0.286,WV=0.714):
    """
    Funcion para calcular DL a partir de un redshift y ciertos parametros cosmologicos
    Si ningun parametros cosmologico es ingresado, se usan los por defecto de https://www.astro.ucla.edu/~wright/CosmoCalc.html

    """
    c = 299792.458 # Velocidad de la luz en km/s
    h = H0/100.
    WR = 4.165E-5/(h*h)   # includes  3 massless neutrino species, T0 = 2.72528
    WK = 1-WM-WR-WV
    az = 1.0/(1+1.0*z)
    age = 0.
    n=1000         # number of points in integrals
    DCMR = 0.0
    
    # do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
    for i in range(n):
        a = az+(1-az)*(i+0.5)/n
        adot = sqrt(WK+(WM/a)+(WR/(a*a))+(WV*a*a))
        #DTT = DTT + 1./adot
        DCMR = DCMR + 1./(a*adot)

    DCMR = (1.-az)*DCMR/n
    ratio = 1.00
    x = sqrt(abs(WK))*DCMR #si el universo es plano x=0 y no entra, por lo tanto el ratio es 1, ya que "y" tambien seria 0                             
    if x > 0.1:
        if WK > 0:
            ratio =  0.5*(exp(x)-exp(-x))/x 
        else:
            ratio = sin(x)/x
    else:
        y = x*x
    if WK < 0: y = -y
    ratio = 1. + y/6. + y*y/120.

    DCMT = ratio*DCMR
    DA = az*DCMT
    DL = DA/(az*az)
    DL_Mpc = (c/H0)*DL
    return DL_Mpc

In [ ]:
dic_constantes={
 'cteB': 6.460803024157998e-09,
 'cteV': 3.67558126207669e-09,
 'cteR': 2.2319872881446082e-09,
 'cteI': 1.1769629663231628e-09,
 'cteU': 4.346809582387867e-09,
 'cteu': 9.512689427618875e-09,
 'cteg': 4.791168585588861e-09,
 'cter': 2.818556152283821e-09,
 'ctei': 1.9069984363571034e-09,
 'ctez': 1.413365541656362e-09
}

'''
#pasamos a mag
arr_ctes=['cteB','cteV','cteR','cteI','cteU','cteu','cteg','cter','ctei','ctez']
arr_val_ctes=[cteB,cteV,cteR,cteI,cteU,cteu,cteg,cter,ctei,ctez]
V=filtros[q]
constante='cte'+V #pongo V pero es cualquier banda
#print (constante)
for jj in range(len(arr_ctes)): 
    if constante==arr_ctes[jj]: #recorremos el arreglo de constantes en busca de la que le corresponde
        #print (constante,arr_ctes[jj],arr_val_ctes[jj])
        mul=arr_val_ctes[jj]
    
mag=-2.5*np.log10(syn/mul)

'''